In [ ]:
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
# pip install tqdm
# pip install ipywidgets
# pip install "numpy<2.0"
# pip install --upgrade scipy scikit-learn
# pip install pyarrow pandas
# pip install lightgbm matplotlib
# pip install optuna
# pip install -U sentence-transformers
# pip install \
#     --extra-index-url=https://pypi.nvidia.com \
#     "cudf-cu12==25.6." "dask-cudf-cu12==25.6." "cuml-cu12==25.6.*" \
#     "cugraph-cu12==25.6." "nx-cugraph-cu12==25.6." "cuxfilter-cu12==25.6.*" \
#     "cucim-cu12==25.6." "pylibraft-cu12==25.6." "raft-dask-cu12==25.6.*" \
#     "cuvs-cu12==25.6." "nx-cugraph-cu12==25.6."
# curl -L https://lambdalabs-guest-agent.s3.us-west-2.amazonaws.com/scripts/install.sh | sudo bash

In [ ]:
# import pandas as pd
# import warnings
# import numpy as np
# import os
# import gc
# import time

# # --- Configuration ---
# warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# # --- File Paths ---
# # Assumes data is in the same directory as the scriptQ
# RAW_DATA_PATHS = {
#     "train": "train_data.parquet",
#     "test": "test_data_r3.parquet",
#     "add_trans": "add_trans.parquet",
#     "add_event": "add_event.parquet",
#     "offer_meta": "offer_metadata.parquet",
# }
# OUTPUT_DIR = "inter"
# OUTPUT_PATHS = {
#     "train": os.path.join(OUTPUT_DIR, "train_enriched.parquet"),
#     "test": os.path.join(OUTPUT_DIR, "test_enriched.parquet"),
# }

# # --- Helper Functions ---


# def load_raw_data(paths):
#     """Loads all necessary raw data files from parquet format."""
#     print("1/7: Loading all raw data files...")
#     data = {}
#     try:
#         for name, path in paths.items():
#             print(f"  -> Loading {path}...")
#             data[name] = pd.read_parquet(path)
#     except FileNotFoundError as e:
#         print(f"❌ ERROR: Could not find a required data file. Details: {e}")
#         print(
#             "Please ensure all .parquet files are in the same directory as the script."
#         )
#         exit()
#     return data


# def combine_and_preprocess(train_df, test_df):
#     """Combines train and test sets and performs initial preprocessing."""
#     print("2/7: Combining data and processing timestamps...")
#     train_len = len(train_df)

#     all_data = pd.concat([train_df, test_df], ignore_index=True)
#     del train_df, test_df
#     gc.collect()

#     # Convert main interaction timestamp
#     all_data["id4"] = pd.to_datetime(all_data["id4"])

#     return all_data, train_len


# def engineer_offer_features(all_data, offer_meta):
#     """Engineers static and basic NLP features from offer metadata."""
#     print("3/7: Engineering static and basic NLP offer features...")

#     # --- Timestamp and Duration Features ---
#     offer_meta["id12"] = pd.to_datetime(offer_meta["id12"])
#     offer_meta["id13"] = pd.to_datetime(offer_meta["id13"])
#     offer_meta["offer_duration_days"] = (
#         offer_meta["id13"] - offer_meta["id12"]
#     ).dt.days
#     offer_meta["offer_start_dayofweek"] = offer_meta["id12"].dt.dayofweek

#     # --- Categorical and Renaming ---
#     offer_meta["brand_id"], _ = pd.factorize(offer_meta["id11"])
#     offer_meta["industry_id"], _ = pd.factorize(offer_meta["id10"])
#     offer_meta.rename(
#         columns={
#             "f375": "redemption_frequency",
#             "f376": "discount_rate",
#             "id8": "member_industry_code",
#             "id9": "offer_name",
#             "f378": "offer_body",
#         },
#         inplace=True,
#     )
#     offer_meta["is_industry_match"] = (
#         offer_meta["industry_id"] == offer_meta["member_industry_code"]
#     ).astype(int)

#     # --- Basic NLP Features ---
#     print("  -> Generating basic NLP features from offer text...")
#     offer_meta["offer_name"] = (
#         offer_meta["offer_name"].astype(str).str.lower().fillna("")
#     )
#     offer_meta["offer_body"] = (
#         offer_meta["offer_body"].astype(str).str.lower().fillna("")
#     )
#     offer_meta["offer_name_len"] = offer_meta["offer_name"].str.len()
#     offer_meta["offer_body_words"] = offer_meta["offer_body"].str.split().str.len()

#     # Keyword matching
#     offer_meta["has_keyword_cashback"] = (
#         offer_meta["offer_body"]
#         .str.contains("cash back|statement credit|refund", regex=True)
#         .astype(int)
#     )
#     offer_meta["has_keyword_points"] = (
#         offer_meta["offer_body"]
#         .str.contains("points|miles|rewards", regex=True)
#         .astype(int)
#     )
#     offer_meta["has_keyword_discount"] = (
#         offer_meta["offer_body"]
#         .str.contains("% off|discount|save", regex=True)
#         .astype(int)
#     )
#     offer_meta["has_keyword_spend_x"] = (
#         offer_meta["offer_body"].str.contains("spend", regex=False).astype(int)
#     )

#     # --- Merge into Main DataFrame ---
#     offer_features_to_merge = [
#         "id3",
#         "offer_duration_days",
#         "offer_start_dayofweek",
#         "brand_id",
#         "industry_id",
#         "redemption_frequency",
#         "discount_rate",
#         "is_industry_match",
#         "offer_name_len",
#         "offer_body_words",
#         "has_keyword_cashback",
#         "has_keyword_points",
#         "has_keyword_discount",
#         "has_keyword_spend_x",
#         "id12",
#         "id13",  # Keep timestamps for interaction feature creation
#     ]

#     all_data["id3"] = all_data["id3"].astype("int64")
#     offer_meta["id3"] = offer_meta["id3"].astype("int64")
#     all_data = all_data.merge(offer_meta[offer_features_to_merge], on="id3", how="left")

#     # Create time-based interaction features
#     all_data["days_since_offer_start"] = (all_data["id4"] - all_data["id12"]).dt.days
#     all_data["days_until_offer_end"] = (all_data["id13"] - all_data["id4"]).dt.days
#     all_data.drop(columns=["id12", "id13"], inplace=True)

#     return all_data


# def engineer_event_history_features(all_data, add_event, offer_meta):
#     """Engineers historical customer-offer interaction features, preventing data leakage."""
#     print("4/7: Engineering customer-offer historical interaction features...")

#     # --- Preprocessing ---
#     add_event["id4"] = pd.to_datetime(add_event["id4"])
#     add_event["id7"] = pd.to_datetime(
#         add_event["id7"], errors="coerce"
#     )  # Click timestamp can be NaT
#     add_event["id3"] = add_event["id3"].astype("int64")

#     # Merge brand/industry info needed for grouped features
#     add_event = add_event.merge(
#         offer_meta[["id3", "brand_id", "industry_id"]], on="id3", how="left"
#     )
#     add_event.sort_values(by=["id2", "id4"], inplace=True)

#     # --- Feature Creation ---
#     add_event["clicked"] = add_event["id7"].notna().astype(int)

#     # Time-since-last-event features
#     add_event["time_since_last_impression_seconds"] = (
#         add_event.groupby("id2")["id4"].diff().dt.total_seconds()
#     )
#     add_event["click_timestamp"] = add_event["id4"].where(add_event["clicked"] == 1)
#     add_event["last_click_timestamp"] = add_event.groupby("id2")[
#         "click_timestamp"
#     ].ffill()
#     add_event["time_since_last_click_seconds"] = (
#         add_event["id4"] - add_event["last_click_timestamp"]
#     ).dt.total_seconds()

#     # CRITICAL: Lagged cumulative counts to prevent data leakage from the current event
#     add_event["customer_total_impressions_before"] = add_event.groupby("id2").cumcount()
#     add_event["customer_brand_impressions_before"] = add_event.groupby(
#         ["id2", "brand_id"]
#     ).cumcount()
#     add_event["customer_total_clicks_before"] = (
#         add_event.groupby("id2")["clicked"].cumsum().shift(1).fillna(0)
#     )
#     add_event["customer_brand_clicks_before"] = (
#         add_event.groupby(["id2", "brand_id"])["clicked"].cumsum().shift(1).fillna(0)
#     )

#     # Historical Click-Through-Rates (CTR)
#     epsilon = 1e-6
#     add_event["customer_ctr_before"] = add_event["customer_total_clicks_before"] / (
#         add_event["customer_total_impressions_before"] + epsilon
#     )
#     add_event["customer_brand_ctr_before"] = add_event[
#         "customer_brand_clicks_before"
#     ] / (add_event["customer_brand_impressions_before"] + epsilon)

#     # --- Merge into Main DataFrame ---
#     event_key = ["id2", "id3", "id4"]
#     event_features_to_merge = [
#         "time_since_last_impression_seconds",
#         "time_since_last_click_seconds",
#         "customer_total_impressions_before",
#         "customer_brand_impressions_before",
#         "customer_total_clicks_before",
#         "customer_brand_clicks_before",
#         "customer_ctr_before",
#         "customer_brand_ctr_before",
#     ]

#     for col in event_key:
#         all_data[col] = all_data[col].astype(add_event[col].dtype)

#     all_data = all_data.merge(
#         add_event[event_key + event_features_to_merge], on=event_key, how="left"
#     )

#     return all_data


# def engineer_transaction_history_features(all_data, add_trans):
#     """Engineers point-in-time customer transaction features using merge_asof."""
#     print("5/7: Engineering POINT-IN-TIME customer transaction features...")

#     # --- Preprocessing ---
#     add_trans["f370"] = pd.to_datetime(add_trans["f370"])
#     time_as_seconds = pd.to_numeric(add_trans["f371"], errors="coerce")
#     add_trans["transaction_timestamp"] = add_trans["f370"] + pd.to_timedelta(
#         time_as_seconds, unit="s"
#     )
#     add_trans.sort_values(by=["id2", "transaction_timestamp"], inplace=True)

#     # --- Build Point-in-Time Aggregates ---
#     print("  -> Calculating point-in-time cumulative aggregates...")
#     trans_history = add_trans[
#         ["id2", "transaction_timestamp", "f367", "f368", "f369"]
#     ].copy()

#     is_debit = trans_history["f369"] == "D"
#     debit_amount = trans_history["f367"].where(is_debit, 0)
#     credit_amount = trans_history["f367"].where(trans_history["f369"] == "C", 0)

#     grouped = trans_history.groupby("id2")
    
#     # === FIX (Warning): Added include_groups=False to silence FutureWarning ===
#     trans_history["customer_total_spend"] = grouped.apply(
#         lambda x: debit_amount[x.index].cumsum(), include_groups=False
#     ).reset_index(level=0, drop=True)
#     trans_history["customer_total_refund_amount"] = grouped.apply(
#         lambda x: credit_amount[x.index].cumsum(), include_groups=False
#     ).reset_index(level=0, drop=True)
#     trans_history["customer_num_transactions"] = grouped.cumcount() + 1
#     trans_history["customer_num_debits"] = grouped.apply(
#         lambda x: (x["f369"] == "D").cumsum(), include_groups=False
#     ).reset_index(level=0, drop=True)
#     trans_history["customer_num_refunds"] = grouped.apply(
#         lambda x: (x["f369"] == "C").cumsum(), include_groups=False
#     ).reset_index(level=0, drop=True)

#     # Forward-fill to handle groups with no debits/credits initially
#     for col in [
#         "customer_total_spend",
#         "customer_total_refund_amount",
#         "customer_num_debits",
#         "customer_num_refunds",
#     ]:
#         trans_history[col] = grouped[col].ffill().fillna(0)

#     epsilon = 1e-6
#     trans_history["customer_avg_trans_amount"] = trans_history[
#         "customer_total_spend"
#     ] / (trans_history["customer_num_debits"] + epsilon)
#     trans_history["customer_refund_rate"] = trans_history["customer_num_refunds"] / (
#         trans_history["customer_num_debits"] + epsilon
#     )

#     # === FIX (Error): Correctly apply expanding nunique on a grouped object ===
#     # trans_history["customer_num_unique_products"] = grouped["f368"].apply(
#     #     lambda s: pd.Series(pd.factorize(s)[0], index=s.index).expanding().nunique()
#     # ).reset_index(level=0, drop=True)
#     # === END FIX (Error) ===

#     # --- Merge into Main DataFrame using merge_asof ---
#     print("  -> Merging point-in-time features into main dataset...")
#     all_data.sort_values(by="id4", inplace=True)
#     trans_history.dropna(subset=["id2", "transaction_timestamp"], inplace=True)

#     features_to_merge = [
#         "id2",
#         "transaction_timestamp",
#         "customer_total_spend",
#         "customer_num_transactions",
#         "customer_avg_trans_amount",
#         # "customer_num_unique_products",
#         "customer_total_refund_amount",
#         "customer_num_refunds",
#         "customer_refund_rate",
#     ]

#     all_data = pd.merge_asof(
#         all_data,
#         trans_history[features_to_merge],
#         left_on="id4",
#         right_on="transaction_timestamp",
#         by="id2",
#         direction="backward",  # CRITICAL for point-in-time correctness
#     )

#     all_data["days_since_last_transaction"] = (
#         all_data["id4"] - all_data["transaction_timestamp"]
#     ).dt.days
#     all_data.drop(columns=["transaction_timestamp"], inplace=True)

#     return all_data


# def finalize_and_impute(all_data, train_len):
#     """Splits data back to train/test and handles NaN imputation."""
#     print("6/7: Finalizing datasets and imputing missing values...")

#     train_df = all_data.iloc[:train_len].copy()
#     test_df = all_data.iloc[train_len:].copy()
#     del all_data
#     gc.collect()

#     # Define columns for each imputation strategy
#     time_cols = [c for c in train_df.columns if "seconds" in c or "days" in c]
#     rate_cols = [c for c in train_df.columns if "rate" in c or "ctr" in c]
#     count_cols = [
#         c
#         for c in train_df.columns
#         if "num_" in c
#         or "total_" in c
#         or "_before" in c
#         or "_spend" in c
#         or "_amount" in c
#     ]
#     id_cols = [c for c in train_df.columns if "brand_id" in c or "industry_id" in c]

#     # Get remaining numeric columns for default filling
#     all_filled_cols = set(time_cols + rate_cols + count_cols + id_cols)
#     other_numeric_cols = [
#         c
#         for c in train_df.select_dtypes(include=np.number).columns
#         if c not in all_filled_cols and c not in ["id", "id2", "id3", "y"]
#     ]

#     print("  -> Imputing NaN values with strategy-based fillers...")
#     for df in [train_df, test_df]:
#         df[time_cols] = df[time_cols].fillna(-999)
#         df[rate_cols] = df[rate_cols].fillna(0.0)
#         df[count_cols] = df[count_cols].fillna(0)
#         df[id_cols] = df[id_cols].fillna(-1)
#         df[other_numeric_cols] = df[other_numeric_cols].fillna(-1)

#     return train_df, test_df


# def save_data(train_df, test_df, paths):
#     """Saves the final dataframes to parquet files."""
#     print("7/7: Saving final datasets to disk...")
#     os.makedirs(os.path.dirname(paths["train"]), exist_ok=True)
#     train_df.to_parquet(paths["train"], index=False)
#     test_df.to_parquet(paths["test"], index=False)
#     print(f"✅ Saved enriched train data to '{paths['train']}'")
#     print(f"✅ Saved enriched test data to '{paths['test']}'")


# def main():
#     """Main function to run the entire feature engineering pipeline."""
#     print("--- Starting Advanced Feature Engineering Pipeline (Fixed) ---")
#     start_time = time.time()

#     # 1. Load Data
#     raw_data = load_raw_data(RAW_DATA_PATHS)

#     # 2. Combine & Preprocess
#     all_data, train_len = combine_and_preprocess(raw_data["train"], raw_data["test"])

#     # 3. Engineer Offer Features
#     all_data = engineer_offer_features(all_data, raw_data["offer_meta"])

#     # 4. Engineer Event History Features
#     all_data = engineer_event_history_features(
#         all_data, raw_data["add_event"], raw_data["offer_meta"]
#     )
#     del raw_data["add_event"]
#     gc.collect()

#     # 5. Engineer Transaction History Features
#     all_data = engineer_transaction_history_features(all_data, raw_data["add_trans"])
#     del raw_data["add_trans"]
#     gc.collect()

#     # 6. Finalize, Impute, and Split
#     train_enriched, test_enriched = finalize_and_impute(all_data, train_len)

#     # 7. Save Data
#     save_data(train_enriched, test_enriched, OUTPUT_PATHS)

#     end_time = time.time()
#     print(f"\n--- Initial Feature Engineering Complete! ---")
#     print(f"Total time taken: {((end_time - start_time) / 60):.2f} minutes")


# if __name__ == "__main__":
#     main()


In [ ]:
# import pyarrow.parquet as pq
# import pandas as pd
# import warnings
# import numpy as np
# import os
# import gc
# import time

# def run_stage_1_train_val_split():
#     TRAIN_INPUT = "inter/train_enriched.parquet"
#     TEST_INPUT = "inter/test_enriched.parquet"
#     INTER_DIR = "inter"
#     BATCH_SIZE = 100_000
#     VAL_RATIO = 0.15

#     print("\n**Script 0: Train-Val Split (Customer-Aware & Time-Based)**")
#     os.makedirs(INTER_DIR, exist_ok=True)

#     parquet_file = pq.ParquetFile(TRAIN_INPUT)

#     # 1. Load IDs and timestamps to determine the split
#     print("1/5: Loading IDs and timestamps to determine validation customers...")
#     id_chunks = []
#     # CRITICAL FIX: Load customer ID 'id2' to ensure customer-based split
#     for batch in parquet_file.iter_batches(
#         batch_size=BATCH_SIZE, columns=["id2", "id4"]
#     ):
#         chunk_df = batch.to_pandas()
#         id_chunks.append(chunk_df)

#     id_df = pd.concat(id_chunks, ignore_index=True)
#     print(f"1/5 ID index DataFrame created with shape: {id_df.shape}")
#     del id_chunks
#     gc.collect()

#     id_df["id4"] = pd.to_datetime(id_df["id4"])
#     id_df.sort_values(by="id4", inplace=True)

#     # 2. Identify validation customers based on time
#     # This ensures that the validation set is from a later time period than the training set.
#     n_rows = len(id_df)
#     split_idx = int(n_rows * (1 - VAL_RATIO))
#     valid_customer_ids = set(id_df["id2"].iloc[split_idx:])
#     print(
#         f"2/5: Identified {len(valid_customer_ids)} customers for the time-based validation set."
#     )
#     del id_df
#     gc.collect()

#     # 3. Stream data and split into train/valid based on the identified customer IDs
#     print("3/5: Splitting full dataset based on validation customer IDs...")
#     train_rows = []
#     valid_rows = []

#     for batch in parquet_file.iter_batches(batch_size=BATCH_SIZE):
#         batch_df = batch.to_pandas()
#         is_valid_customer = batch_df["id2"].isin(valid_customer_ids)
#         valid_rows.append(batch_df[is_valid_customer])
#         train_rows.append(batch_df[~is_valid_customer])
#     del parquet_file, valid_customer_ids

#     # 4. Save the split datasets
#     train_split = pd.concat(train_rows, ignore_index=True)
#     train_split.to_parquet(f"{INTER_DIR}/train_0.parquet", index=False)
#     print(f"3/5 Saved train_0.parquet with shape: {train_split.shape}")
#     del train_split, train_rows
#     gc.collect()

#     valid_split = pd.concat(valid_rows, ignore_index=True)
#     valid_split.to_parquet(f"{INTER_DIR}/valid_0.parquet", index=False)
#     print(f"4/5 Saved valid_0.parquet with shape: {valid_split.shape}")
#     del valid_split, valid_rows
#     gc.collect()

#     # 5. Copy test data for consistency
#     test_df = pd.read_parquet(TEST_INPUT)
#     test_df.to_parquet(f"{INTER_DIR}/test_0.parquet", index=False)
#     print(f"5/5 Saved test_0.parquet with shape: {test_df.shape}")
#     del test_df
#     gc.collect()


# if __name__ == "__main__":
#     # Stage 1: Split enriched training data into train/validation sets
#     run_stage_1_train_val_split()


In [ ]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# --- Configuration ---
DATA_DICT_PATH = "data_dictionary.csv"
INPUT_DIR = "inter"
OUTPUT_DIR = "inter"


def downcast_numerical_dtypes(df):
    """
    Intelligently downcasts numerical columns to save memory.
    Finds float64 and int64 columns and converts them to the smallest possible format.
    """
    print("   -> Downcasting numerical dtypes for memory optimization...")
    # Downcast floats
    float_cols = df.select_dtypes(include=["float64"]).columns
    for col in float_cols:
        df[col] = df[col].astype(np.float32)

    # Downcast integers
    int_cols = df.select_dtypes(include=["int64"]).columns
    for col in int_cols:
        # np.iinfo provides the min/max values for integer types
        if (
            df[col].min() >= np.iinfo(np.int8).min
            and df[col].max() <= np.iinfo(np.int8).max
        ):
            df[col] = df[col].astype(np.int8)
        elif (
            df[col].min() >= np.iinfo(np.int16).min
            and df[col].max() <= np.iinfo(np.int16).max
        ):
            df[col] = df[col].astype(np.int16)
        elif (
            df[col].min() >= np.iinfo(np.int32).min
            and df[col].max() <= np.iinfo(np.int32).max
        ):
            df[col] = df[col].astype(np.int32)
    return df


def prepare_data_for_final_fe(train_df, valid_df, test_df):
    """
    A preparatory cleaning function to run before the main feature engineering.
    1. Removes empty columns.
    2. Converts datetimes.
    3. Downcasts numerical dtypes to save memory.
    4. Converts string categoricals to the memory-efficient 'category' dtype WITHOUT label encoding.
    """
    print("\n--- Step 1: Removing 100% Empty Features ---")

    # Identify and drop columns that are completely null in the training data
    uniques_per_col = train_df.nunique(dropna=True)
    empty_cols = uniques_per_col[uniques_per_col == 0].index.tolist()

    if empty_cols:
        print(f"  -> Found {len(empty_cols)} empty columns to drop: {empty_cols}")
        for df in [train_df, valid_df, test_df]:
            df.drop(columns=empty_cols, inplace=True)
    else:
        print("  -> No completely empty columns found in the training set.")

    # --- Step 2: Apply safe type conversions ---
    print("\n--- Step 2: Applying Safe Type Conversions ---")

    for df in tqdm([train_df, valid_df, test_df], desc="Processing DataFrames"):
        # Convert datetime columns
        for col in ["id4", "id5"]:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])

        # Optimize memory usage by downcasting numerical columns
        df = downcast_numerical_dtypes(df)

        # Convert high-cardinality string columns to 'category' dtype for memory efficiency.
        # CRITICAL: We do NOT apply .cat.codes here. This preserves the original string values
        # for the next feature engineering step, while still saving memory.
        print("   -> Converting string columns to memory-efficient 'category' dtype...")
        object_cols = df.select_dtypes(include=["object"]).columns
        for col in object_cols:
            df[col] = df[col].astype("category")

    print("\n✅ Preparatory cleaning and type casting complete.")
    return train_df, valid_df, test_df


def run_preparatory_preprocessing():
    """
    Main execution block for this preparatory step.
    """
    print("\n** Running Preparatory Preprocessing Script **")
    print("   (To clean and optimize dtypes before final feature engineering)")

    # Load the split data from the previous step
    train_df = pd.read_parquet(f"{INPUT_DIR}/train_0.parquet")
    valid_df = pd.read_parquet(f"{INPUT_DIR}/valid_0.parquet")
    test_df = pd.read_parquet(f"{INPUT_DIR}/test_0.parquet")

    # Run the preparatory function
    processed_train, processed_valid, processed_test = prepare_data_for_final_fe(
        train_df, valid_df, test_df
    )

    # Save the prepared files, ready for the Final FE script
    # These files have the same features, just cleaner and more memory-efficient.
    processed_train.to_parquet(f"{OUTPUT_DIR}/train_1_prepared.parquet")
    processed_valid.to_parquet(f"{OUTPUT_DIR}/valid_1_prepared.parquet")
    processed_test.to_parquet(f"{OUTPUT_DIR}/test_1_prepared.parquet")

    print(
        f"\nSaved prepared data to '{OUTPUT_DIR}/train_1_prepared.parquet' and corresponding files."
    )

    del train_df, valid_df, test_df, processed_train, processed_valid, processed_test
    gc.collect()


if __name__ == "__main__":
    run_preparatory_preprocessing()


In [ ]:
# import pandas as pd
# import warnings
# import numpy as np
# import os
# import gc
# import shutil
# import uuid
# import traceback
# from multiprocessing import Pool, cpu_count
# from functools import partial
# from tqdm import tqdm
# import pyarrow.parquet as pq
# import pyarrow as pa
# import pyarrow.dataset as ds
# from scipy.fft import rfft
# from scipy.stats import entropy
# import scipy.stats

# # --- Configuration ---
# warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
# warnings.filterwarnings('ignore', category=RuntimeWarning)

# # --- Script Settings ---
# SKIP_TRAINING = False
# DATA_DICT_PATH = "data_dictionary.csv"
# INPUT_DIR = "inter"
# OUTPUT_DIR = "inter"
# N_CORES = cpu_count()

# # --- Temporary Directories for Parallel Processing ---
# TEMP_SPLIT_DIR = os.path.join(OUTPUT_DIR, "temp_splits")
# TEMP_PARTS_DIR = os.path.join(OUTPUT_DIR, "temp_parts")
# # NOTE: The pre-merging step and its directory have been removed.

# # --- Helper Functions ---
# def downcast_dtypes(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Downcasts float64 columns to the more memory-efficient float32 type.
#     """
#     for col in df.select_dtypes(include=['float64']).columns:
#         df[col] = df[col].astype(np.float32)
#     return df

# # --- Worker for Phase 1 (Copy-on-Write) ---
# def split_customer_data_worker_global(customer_id_chunk, output_dir):
#     """
#     Takes a chunk of customer IDs, filters the global main DataFrame for those customers,
#     and saves each customer's data to a separate file.
#     """
#     try:
#         sub_df = main_df[main_df['id2'].isin(customer_id_chunk)]
#         for customer_id, group_df in sub_df.groupby('id2'):
#             output_file = os.path.join(output_dir, f"customer_{customer_id}.parquet")
#             group_df.to_parquet(output_file)
#         return True
#     except Exception as e:
#         print(f"Error in splitting worker for chunk starting with {customer_id_chunk[0]}: {e}")
#         return False

# # --- Worker for Phase 2 - Parallel Feature Engineering ---
# def process_customer_file(customer_id, input_dir, output_dir, numerical_cols):
#     """
#     Processes a SINGLE customer's data by reading it from a pre-split file,
#     calculates complex sequential features, and saves the result.
#     """
#     try:
#         input_path = os.path.join(input_dir, f"customer_{customer_id}.parquet")
#         group_df = pd.read_parquet(input_path)

#         def clean_mixed_type(x):
#             if isinstance(x, (list, tuple, np.ndarray)):
#                 return x[0] if len(x) > 0 else np.nan
#             return x

#         for col in group_df.select_dtypes(include=['object']).columns:
#             group_df[col] = group_df[col].apply(clean_mixed_type)

#         f_cols = [c for c in group_df.columns if c.startswith('f')]
#         if f_cols:
#             group_df[f_cols] = group_df[f_cols].apply(pd.to_numeric, errors='coerce')

#         group_df = group_df.copy()

#         def wma(series, period):
#             return series.ewm(alpha=2 / (period + 1), adjust=False).mean()

#         def hma(series, period):
#             if period <= 1: return series
#             period_sqrt = max(1, int(np.sqrt(period)))
#             wma1 = 2 * wma(series, period // 2)
#             wma2 = wma(series, period)
#             return wma(wma1 - wma2, period_sqrt)

#         def safe_shannon_entropy(series):
#             if len(series) < 15: return np.nan
#             interpolated_series = series.interpolate(method='linear').ffill().bfill()
#             if interpolated_series.isnull().any() or interpolated_series.nunique() < 2: return np.nan
#             psd = np.abs(rfft(interpolated_series.values))**2
#             if psd.sum() == 0: return np.nan
#             psd_prob = psd / psd.sum()
#             return scipy.stats.entropy(psd_prob)

#         days_since_last_event = (group_df['id4'].diff().dt.total_seconds() / (3600 * 24)) + 1e-6
#         days_since_last_event = pd.to_numeric(days_since_last_event, errors='coerce').fillna(1e-6)

#         is_click = (group_df['f29'].diff() > 0) if 'f29' in group_df.columns else pd.Series([False] * len(group_df), index=group_df.index)
#         group_df['is_click_int'] = is_click.astype(int)

#         group_df['offer_interaction_count'] = group_df.groupby('id3').cumcount()
#         if 'offer_category' in group_df.columns:
#             group_df['category_interaction_count'] = group_df.groupby('offer_category').cumcount()
#             category_clicks = group_df.groupby('offer_category')['is_click_int'].cumsum().shift(1).fillna(0)
#             category_views = group_df.groupby('offer_category').cumcount()
#             group_df['customer_category_ctr'] = category_clicks / (category_views + 1)

#         if 'session_id' in group_df.columns:
#             session_gb = group_df.groupby('session_id')
#             group_df['session_offer_count'] = session_gb.cumcount()
#             group_df['session_clicks'] = session_gb['is_click_int'].cumsum().shift(1).fillna(0)
#             group_df['time_since_session_start_mins'] = (group_df['id4'] - session_gb['id4'].transform('min')).dt.total_seconds() / 60

#         if 'offer_age_days' in group_df.columns:
#             group_df['offer_age_days'] = pd.to_numeric(group_df['offer_age_days'], errors='coerce')
#             rolling_avg_age = group_df['offer_age_days'].rolling(10, min_periods=1).mean()
#             group_df['age_vs_recent_avg'] = group_df['offer_age_days'] - rolling_avg_age

#         group_df['time_since_last_event_hours'] = days_since_last_event * 24
#         if 'f29' in group_df.columns:
#             click_timestamps = group_df['id4'].where(is_click).ffill()
#             group_df['time_since_last_click'] = (group_df['id4'] - click_timestamps).dt.total_seconds() / 3600

#         key_velocity_cols = {'f43': 'balance', 'f77': 'engagement_ratio', 'f59': 'time_spent'}
#         for col, name in key_velocity_cols.items():
#             if col in group_df.columns:
#                 s_numeric = group_df[col]
#                 if s_numeric.isnull().all():
#                     group_df[f'{name}_velocity'] = np.nan
#                     continue
#                 change = s_numeric.diff()
#                 velocity = change / days_since_last_event
#                 lower_q = velocity.expanding(min_periods=5).quantile(0.01).ffill()
#                 upper_q = velocity.expanding(min_periods=5).quantile(0.99).ffill()
#                 group_df[f'{name}_velocity'] = velocity.clip(lower_q, upper_q)

#         for col in numerical_cols:
#             if col in group_df.columns:
#                 s_numeric = pd.to_numeric(group_df[col], errors='coerce')
#                 if s_numeric.isnull().all(): continue
#                 group_df[f'{col}_diff1'] = s_numeric.diff(1)
#                 for k in [5, 10, 20]:
#                     group_df[f'{col}hma{k}'] = hma(s_numeric, k)
#                 for k in [10, 20]:
#                     group_df[f'{col}roll_std{k}'] = s_numeric.rolling(k, min_periods=3).std()

#         key_ts_cols = ['f218', 'f219', 'f220', 'f102']
#         for col in key_ts_cols:
#             if col in group_df.columns:
#                 s_numeric = pd.to_numeric(group_df[col], errors='coerce')
#                 if s_numeric.isnull().all(): continue
#                 rm = s_numeric.rolling(5, min_periods=1).mean()
#                 rv = s_numeric.rolling(5, min_periods=1).var()
#                 group_df[f'{col}_stability_5'] = rm.rolling(5, min_periods=1).var()
#                 group_df[f'{col}_lumpiness_5'] = rv.rolling(5, min_periods=1).var()
#                 group_df[f'{col}_rolling_shannon_entropy'] = s_numeric.rolling(20, min_periods=15).apply(safe_shannon_entropy, raw=False)

#         group_df['time_since_last_seen_this_offer'] = group_df.groupby('id3')['id4'].diff().dt.total_seconds() / 3600
#         if 'offer_category' in group_df.columns:
#             group_df['time_since_last_seen_this_category'] = group_df.groupby('offer_category')['id4'].diff().dt.total_seconds() / 3600

#         group_df.drop(columns=['is_click_int', 'session_id'], inplace=True, errors='ignore')
#         group_df = downcast_dtypes(group_df)
#         output_path = os.path.join(output_dir, f"part_{customer_id}.parquet")
#         group_df.to_parquet(output_path)
#         return True
#     except Exception as e:
#         print(f"\n--- CRITICAL ERROR: Processing failed for customer ID: {customer_id} ---")
#         traceback.print_exc()
#         return False

# # --- NEW: OOM-Safe Merge using PyArrow Dataset API ---
# def merge_with_dataset_api(source_dir, output_path, schema, is_test_run=False):
#     """
#     Merges all Parquet files in a directory into a single file using the
#     memory-efficient PyArrow Dataset API.
#     """
#     print(f"\nStep 4/5: Merging files from '{source_dir}' into '{output_path}' using Dataset API...")
    
#     # Define the columns to write, excluding 'y' for test runs
#     columns_to_write = schema.names
#     if is_test_run and 'y' in columns_to_write:
#         print(" -> Test run detected. Excluding 'y' column from final output.")
#         columns_to_write.remove('y')

#     # Create a dataset from the directory of part files
#     dataset = ds.dataset(source_dir, format="parquet", schema=schema)
    
#     # Create a scanner to select columns. This is a lazy operation.
#     scanner = dataset.scanner(columns=columns_to_write)
    
#     # Use a temporary file for writing to avoid conflicts if the script is re-run
#     temp_output_file = output_path + ".tmp"

#     # Write the dataset to a single new Parquet file
#     ds.write_dataset(
#         scanner,
#         base_dir=os.path.dirname(temp_output_file),
#         basename_template=os.path.basename(temp_output_file),
#         format="parquet",
#         max_partitions=1, # Force writing to a single file
#         existing_data_behavior='overwrite_or_ignore'
#     )
    
#     # Rename the temporary file to the final output path
#     if os.path.exists(output_path):
#         os.remove(output_path)
#     os.rename(temp_output_file, output_path)

#     print(f" -> Successfully merged files.")

# # --- Main Feature Engineering Function ---
# def engineer_features_final(
#     df: pd.DataFrame,
#     output_path: str,
#     data_dict_path: str,
#     global_start_time=None,
#     offer_profile_features=None,
#     master_schema=None,
#     is_test_run=False
# ):
#     print(f"\nProcessing DataFrame with shape: {df.shape} to create file: {output_path}")
#     print(f"Using {N_CORES} cores for parallel steps.")

#     for temp_dir in [TEMP_SPLIT_DIR, TEMP_PARTS_DIR]:
#         if os.path.exists(temp_dir): shutil.rmtree(temp_dir)
#         os.makedirs(temp_dir)
#         print(f"  -> Created temporary directory: {temp_dir}")

#     data_dict = pd.read_csv(data_dict_path)
#     type_map = {row["masked_column"]: row["Type"].strip() for _, row in data_dict.iterrows()}
#     numerical_cols = [col for col in df.columns if col.startswith("f") and type_map.get(col) == "Numerical"]

#     print("Step 1/5: Engineering fast, vectorized features...")
#     df["id4"] = pd.to_datetime(df["id4"])
#     df = df.sort_values(by=['id2', 'id4'])
#     time_diff_mins = df.groupby('id2')['id4'].diff().dt.total_seconds().div(60)
#     session_break = (time_diff_mins > 30).cumsum()
#     df['session_id'] = df['id2'].astype(str) + '_' + session_break.astype(str)
#     df.rename(columns={'f223': 'offer_age_days', 'f224': 'offer_time_to_expiry_days'}, inplace=True)

#     df['customer_account_age_days'] = (df['id4'] - df.groupby('id2')['id4'].transform('min')).dt.total_seconds() / (3600 * 24)
#     if global_start_time is None: global_start_time = df['id4'].min()
#     df['time_since_dataset_start_days'] = (df['id4'] - global_start_time).dt.total_seconds() / (3600 * 24)

#     category_cols = [f'f{i}' for i in range(226, 233)]; existing_cat_cols = [c for c in category_cols if c in df.columns]
#     if existing_cat_cols:
#         df[existing_cat_cols] = df[existing_cat_cols].apply(pd.to_numeric, errors='coerce')
#         df['offer_category'] = df[existing_cat_cols].idxmax(axis=1)

#     if offer_profile_features is None:
#         print("Step 2/5: Computing offer profiles from training data...")
#         base_offer_profiles = df.groupby('id3').agg(
#             offer_popularity=('id1', 'count'),
#             offer_customer_reach=('id2', 'nunique')
#         )
#         monthly_counts = df.groupby([df['id4'].dt.to_period('M'), 'id3'])['id1'].count().unstack('id3').fillna(0)
        
#         if len(monthly_counts) >= 6:
#             growth_rate = (monthly_counts.iloc[-3:].mean() - monthly_counts.iloc[-6:-3].mean()) / (monthly_counts.iloc[-6:-3].mean() + 1e-6)
#             base_offer_profiles['offer_growth_rate'] = growth_rate
#         else:
#             base_offer_profiles['offer_growth_rate'] = np.nan
            
#         base_offer_profiles['offer_lifecycle_stage'] = pd.cut(base_offer_profiles['offer_growth_rate'], bins=[-np.inf, 0, 0.2, np.inf], labels=[0, 1, 2]).astype('Int8')
#         offer_profile_features = base_offer_profiles
#     else:
#         print("Step 2/5: Using pre-computed offer profiles for validation/test data.")

#     print("Step 2.5/5: Engineering leakage-free historical features...")
#     for col in tqdm(offer_profile_features.columns, desc="    -> Mapping offer profiles"):
#         df[col] = df['id3'].map(offer_profile_features[col])

#     if 'y' in df.columns:
#         df['y'] = pd.to_numeric(df['y'], errors='coerce')

#     if not is_test_run and 'y' in df.columns:
#         offer_group = df.groupby('id3')
#         historical_clicks = offer_group['y'].cumsum().shift(1)
#         historical_views = offer_group.cumcount()
#         df['offer_historical_ctr_fixed'] = (historical_clicks / (historical_views + 1)).fillna(0)
#     else:
#         df['offer_historical_ctr_fixed'] = 0

#     cust_popularity_avg = df.groupby('id2')['offer_popularity'].expanding().mean().shift(1)
#     cust_popularity_avg = cust_popularity_avg.reset_index(level=0, drop=True)
#     df['popularity_vs_customer_norm'] = df['offer_popularity'] - cust_popularity_avg.fillna(df['offer_popularity'])

#     print("Step 2.7/5: Engineering dynamic customer profiles...")
#     all_profile_cols = numerical_cols
#     for col in tqdm(all_profile_cols, desc="    -> Calculating dynamic profiles"):
#         if col in df.columns:
#             customer_group = df.groupby('id2')[col]
#             expanding_mean = customer_group.expanding().mean().shift(1)
#             expanding_std = customer_group.expanding().std().shift(1)
#             expanding_max = customer_group.expanding().max().shift(1)
#             expanding_min = customer_group.expanding().min().shift(1)
#             df[f'dynamic_{col}_mean'] = expanding_mean.reset_index(level=0, drop=True)
#             df[f'dynamic_{col}_std'] = expanding_std.reset_index(level=0, drop=True)
#             df[f'dynamic_{col}_max'] = expanding_max.reset_index(level=0, drop=True)
#             df[f'dynamic_{col}_min'] = expanding_min.reset_index(level=0, drop=True)

#     fill_cols = [c for c in df.columns if c.startswith('dynamic_')]
#     df[fill_cols] = df[fill_cols].fillna(0)

#     print(f"\nStep 3/5: Pre-splitting data for {df['id2'].nunique()} customers in parallel...")
#     global main_df
#     main_df = df
#     customer_ids = main_df['id2'].unique()
#     id_chunks = np.array_split(customer_ids, N_CORES * 8)
#     split_func = partial(split_customer_data_worker_global, output_dir=TEMP_SPLIT_DIR)
#     with Pool(N_CORES) as p:
#         for _ in tqdm(p.imap_unordered(split_func, id_chunks), total=len(id_chunks), desc=" -> Parallel Splitting"):
#             pass
#     del df, main_df; gc.collect()

#     print(f"Step 3.5/5: Discovering and processing customer files...")
#     customer_files = [f for f in os.listdir(TEMP_SPLIT_DIR) if f.startswith('customer_') and f.endswith('.parquet')]
#     customer_ids_from_files = [f.split('_')[1].split('.')[0] for f in customer_files]
#     print(f"    -> Found {len(customer_ids_from_files)} customer files to process.")

#     if customer_ids_from_files:
#         processing_func = partial(process_customer_file, input_dir=TEMP_SPLIT_DIR, output_dir=TEMP_PARTS_DIR, numerical_cols=numerical_cols)
#         with Pool(N_CORES) as p:
#             for _ in tqdm(p.imap_unordered(processing_func, customer_ids_from_files), total=len(customer_ids_from_files), desc=" -> Feature Engineering"):
#                 pass
#     shutil.rmtree(TEMP_SPLIT_DIR)

#     part_files = [os.path.join(TEMP_PARTS_DIR, f) for f in os.listdir(TEMP_PARTS_DIR) if f.endswith('.parquet')]
#     if not part_files:
#         print("Warning: No temporary part files were created.")
#         shutil.rmtree(TEMP_PARTS_DIR)
#         return offer_profile_features, None, global_start_time

#     if master_schema is None:
#         print("     -> No master schema found. Creating a robust union schema from a sample of part files...")
#         sample_files = part_files[:min(200, len(part_files))]
#         all_schemas = [pq.read_schema(f) for f in sample_files]
#         all_fields = {}
#         for schema in all_schemas:
#             for field in schema:
#                 if field.name not in all_fields or pa.types.is_null(all_fields[field.name].type):
#                     all_fields[field.name] = field
#         master_schema = pa.schema(list(all_fields.values()))
#         print("     -> Master schema created successfully.")

#     # --- REVISED STEP 4 & 5: OOM-Safe Merge using PyArrow Dataset API ---
#     merge_with_dataset_api(TEMP_PARTS_DIR, output_path, master_schema, is_test_run)
#     shutil.rmtree(TEMP_PARTS_DIR) # Clean up the thousands of small files

#     print(f"✅ Finished creating final file: {output_path}")
#     return offer_profile_features, master_schema, global_start_time

# # --- Main Execution ---
# if __name__ == "__main__":
#     print("🚀 *** Optimized Production-Grade Feature Engineering Script ***")
#     os.makedirs(OUTPUT_DIR, exist_ok=True)

#     offer_profiles_artifact = None
#     master_schema_artifact = None
#     global_start_time_artifact = None

#     if not SKIP_TRAINING:
#         print("\n--- Processing Training Data ---")
#         train_df = pd.read_parquet(f"{INPUT_DIR}/train_1_prepared.parquet", engine='fastparquet')

#         offer_profiles_artifact, master_schema_artifact, global_start_time_artifact = engineer_features_final(
#             train_df,
#             output_path=f"{OUTPUT_DIR}/train_2_final.parquet",
#             data_dict_path=DATA_DICT_PATH
#         )
#         offer_profiles_artifact.to_parquet(f"{OUTPUT_DIR}/offer_profiles_artifact.parquet")

#         if master_schema_artifact:
#             with open(f"{OUTPUT_DIR}/master_schema.arrow", "wb") as f:
#                 serialized_buffer = master_schema_artifact.serialize()
#                 f.write(serialized_buffer.to_pybytes())

#         del train_df; gc.collect()
#     else:
#         print("\n--- Skipping Training Phase: Loading artifacts from disk... ---")
#         offer_profiles_artifact = pd.read_parquet(f"{OUTPUT_DIR}/offer_profiles_artifact.parquet", engine='fastparquet')
#         with open(f"{OUTPUT_DIR}/master_schema.arrow", "rb") as f:
#             master_schema_artifact = pa.ipc.read_schema(pa.py_buffer(f.read()))

#         train_pq_file = pq.ParquetFile(f"{OUTPUT_DIR}/train_2_final.parquet")
#         global_start_time_artifact = train_pq_file.read(columns=['id4'])['id4'].min().as_py()
#         print(" -> Artifacts loaded successfully.")

#     print("\n--- Processing Validation Data ---")
#     valid_df = pd.read_parquet(f"{INPUT_DIR}/valid_1_prepared.parquet", engine='fastparquet')
#     valid_df.reset_index(drop=True, inplace=True)

#     _, _, _ = engineer_features_final(
#         valid_df,
#         output_path=f"{OUTPUT_DIR}/valid_2_final.parquet",
#         data_dict_path=DATA_DICT_PATH,
#         global_start_time=global_start_time_artifact,
#         offer_profile_features=offer_profile_features,
#         master_schema=master_schema_artifact
#     )
#     del valid_df; gc.collect()

#     print("\n--- Processing Test Data ---")
#     test_file_path = f"{INPUT_DIR}/test_1_prepared.parquet"
#     if os.path.exists(test_file_path):
#         test_df = pd.read_parquet(test_file_path, engine='fastparquet')
#         test_df.reset_index(drop=True, inplace=True)
#         if 'y' not in test_df.columns: test_df['y'] = -1

#         _, _, _ = engineer_features_final(
#             test_df,
#             output_path=f"{OUTPUT_DIR}/test_2_final.parquet",
#             data_dict_path=DATA_DICT_PATH,
#             global_start_time=global_start_time_artifact,
#             offer_profile_features=offer_profiles_artifact,
#             master_schema=master_schema_artifact,
#             is_test_run=True
#         )
#         del test_df, offer_profiles_artifact
#         gc.collect()
#     else:
#         print(f"\n'{test_file_path}' not found. Skipping test set processing.")

#     print("\n--- All datasets have been feature-engineered and saved successfully. ---")


In [ ]:
# import pandas as pd
# import os
# import gc

# print("🚀 --- Starting Data Reconciliation Script --- 🚀")

# # --- Configuration ---
# # Directory where your intermediate files are stored
# INPUT_DIR = "inter" 

# # Base names for the file sets to process
# DATA_SETS = ["train", "valid", "test"]

# for dataset_name in DATA_SETS:
#     print(f"\n--- Processing: {dataset_name} ---")

#     # --- File Paths ---
#     # Path to the clean base file from your new pipeline
#     prepared_path = os.path.join(INPUT_DIR, f"{dataset_name}_1_prepared.parquet")
#     # Path to the feature-rich file from your OLD pipeline's output
#     final_old_path = os.path.join(INPUT_DIR, f"{dataset_name}_2_final.parquet")
#     # Path for the new, reconciled output file. This matches the expected input for the next step.
#     output_path = os.path.join(INPUT_DIR, f"{dataset_name}_2_final.parquet")

#     if not os.path.exists(prepared_path) or not os.path.exists(final_old_path):
#         print(f"  -> ⚠  Skipping {dataset_name}: One or both input files not found.")
#         continue

#     # --- Load DataFrames ---
#     print(f"  -> Loading '{os.path.basename(prepared_path)}'...")
#     df_prepared = pd.read_parquet(prepared_path)
    
#     print(f"  -> Loading '{os.path.basename(final_old_path)}' (from old pipeline)...")
#     df_final_old = pd.read_parquet(final_old_path)

#     print(f"  -> Original shapes: Prepared={df_prepared.shape}, Old Final={df_final_old.shape}")

#     # --- Identify New Columns ---
#     # Find which columns were created in the old advanced feature engineering step.
#     # These are the columns present in the old final file but NOT in the prepared file.
#     prepared_cols = set(df_prepared.columns)
#     final_old_cols = set(df_final_old.columns)
#     new_feature_cols = list(final_old_cols - prepared_cols)
    
#     print(f"  -> Identified {len(new_feature_cols)} new features to merge.")

#     # We need the primary key 'id1' to perform the merge
#     merge_cols = ['id1'] + new_feature_cols
    
#     # --- Perform the Merge ---
#     # We use a left merge to ensure we keep all rows from the clean prepared dataframe
#     # and add the corresponding new features.
#     print("  -> Merging new features onto the prepared base dataframe...")
#     reconciled_df = pd.merge(
#         df_prepared,
#         df_final_old[merge_cols],
#         on='id1',
#         how='left'
#     )

#     print(f"  -> Reconciled shape: {reconciled_df.shape}")

#     # --- Save the Reconciled DataFrame ---
#     # This overwrites the old _2_final file with our new, corrected version.
#     print(f"  -> Saving reconciled data to '{os.path.basename(output_path)}'...")
#     reconciled_df.to_parquet(output_path, index=False)
    
#     # --- Clean up memory ---
#     del df_prepared, df_final_old, reconciled_df
#     gc.collect()

# print("\n✅ --- Reconciliation Complete --- ✅")
# print("You can now SKIP the 'Optimized Production-Grade Feature Engineering' cell and run the subsequent cells.")

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import warnings
from tqdm import tqdm

# --- Configuration ---
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
INPUT_DIR = "inter"
OUTPUT_DIR = "inter"

# --- RE-INTRODUCED: Using your hardcoded list ---
ALL_CATEGORICAL_FEATURES = [
    "brand_id", "industry_id", "member_industry_code", 
    "offer_start_dayofweek", "is_industry_match", "has_keyword_cashback", 
    "has_keyword_points", "has_keyword_discount", "has_keyword_spend_x", 
    "offer_category", "time_of_day_bin", "offer_lifecycle_stage", 
    "is_weekend", "is_holiday_week", "is_payday_week"
]

# --- RE-INTRODUCED: Your memory downcasting function ---
def downcast_numerical_dtypes(df):
    """
    Intelligently downcasts numerical columns to the smallest possible format
    to save memory.
    """
    # Downcast floats
    float_cols = df.select_dtypes(include=["float64"]).columns
    for col in float_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float32)

    # Downcast integers
    int_cols = df.select_dtypes(include=["int64"]).columns
    for col in int_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        if df[col].isnull().any():
            continue
        min_val, max_val = df[col].min(), df[col].max()
        if min_val >= np.iinfo(np.int8).min and max_val <= np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
        elif min_val >= np.iinfo(np.int16).min and max_val <= np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
        elif min_val >= np.iinfo(np.int32).min and max_val <= np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
    return df


def preprocess_for_model(train_df, valid_df, test_df):
    """
    A robust, final preprocessing function using a hybrid approach.
    """
    print("\n--- Running Final Preprocessing for Model Training ---")

    print("Step 1/5: Removing completely empty columns...")
    uniques_per_col = train_df.nunique(dropna=True)
    empty_cols = uniques_per_col[uniques_per_col == 0].index.tolist()
    if empty_cols:
        print(f"  -> Found {len(empty_cols)} empty columns to drop: {empty_cols}")
        for df in [train_df, valid_df, test_df]:
            df.drop(columns=empty_cols, inplace=True, errors='ignore')

    print(f"\nStep 2/5: Applying Label Encoding to {len(ALL_CATEGORICAL_FEATURES)} specified categorical features...")
    for col in tqdm(ALL_CATEGORICAL_FEATURES, desc="Encoding Specified Cats"):
        if col in train_df.columns:
            train_df[col], valid_df[col], test_df[col] = train_df[col].fillna("Missing"), valid_df[col].fillna("Missing"), test_df[col].fillna("Missing")
            learned_categories = train_df[col].astype('category').dtype
            for df in [train_df, valid_df, test_df]:
                df[col] = df[col].astype(learned_categories).cat.codes.astype('int16')

    # --- NEW: Safety net to handle any remaining object columns ---
    print("\nStep 3/5: Catch-all for any remaining object columns...")
    remaining_obj_cols = train_df.select_dtypes(include=['object']).columns.tolist()
    if remaining_obj_cols:
        print(f"  -> Found {len(remaining_obj_cols)} leftover object columns to encode: {remaining_obj_cols}")
        for col in tqdm(remaining_obj_cols, desc="Encoding Leftover Cats"):
            train_df[col], valid_df[col], test_df[col] = train_df[col].fillna("Missing"), valid_df[col].fillna("Missing"), test_df[col].fillna("Missing")
            learned_categories = train_df[col].astype('category').dtype
            for df in [train_df, valid_df, test_df]:
                df[col] = df[col].astype(learned_categories).cat.codes.astype('int16')
    else:
        print("  -> No leftover object columns found. All good.")


    print("\nStep 4/5: Finalizing data (handling infinities and downcasting)...")
    for df in [train_df, valid_df, test_df]:
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        # Using your downcasting function
        df = downcast_numerical_dtypes(df)

    print("\nStep 5/5: Finalizing label 'y' dtype...")
    for df in [train_df, valid_df]:
        if 'y' in df.columns:
            df['y'] = df['y'].astype(np.int8)

    print("\n✅ Final preprocessing is complete.")
    return train_df, valid_df, test_df


def run_final_preprocessing():
    """Main execution block."""
    print("\n** Running Final Hybrid Preprocessing Script **")
    try:
        train_df = pd.read_parquet(f"{INPUT_DIR}/train_2_final.parquet")
        valid_df = pd.read_parquet(f"{INPUT_DIR}/valid_2_final.parquet")
        test_df = pd.read_parquet(f"{INPUT_DIR}/test_2_final.parquet")
    except FileNotFoundError as e:
        print(f"❌ ERROR: Could not find input file. Details: {e}")
        return

    # --- FIX: Remove 'session_id' if it exists ---
    for df in [train_df, valid_df, test_df]:
        if 'session_id' in df.columns:
            print("Removing 'session_id' column...")
            df.drop(columns=['session_id'], inplace=True, errors='ignore')
            break

    processed_train, processed_valid, processed_test = preprocess_for_model(
        train_df, valid_df, test_df
    )

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print("\nSaving model-ready data...")
    # Using engine='fastparquet' and index=False to prevent previous errors
    processed_train.to_parquet(f"{OUTPUT_DIR}/train_3_model_ready.parquet", index=False, engine='fastparquet')
    processed_valid.to_parquet(f"{OUTPUT_DIR}/valid_3_model_ready.parquet", index=False, engine='fastparquet')
    processed_test.to_parquet(f"{OUTPUT_DIR}/test_3_model_ready.parquet", index=False, engine='fastparquet')
    print(f"  -> Saved model-ready data to '{OUTPUT_DIR}/'")

    del train_df, valid_df, test_df, processed_train, processed_valid, processed_test
    gc.collect()


if __name__ == "__main__":
    run_final_preprocessing()

In [ ]:
# =================================================================================
# GPU-Accelerated Feature Selection for Ranking - Refactored and Corrected Script
# =================================================================================

import pandas as pd
import numpy as np
import os
import gc
import json
import warnings
from tqdm import tqdm
import lightgbm as lgb

# Import RAPIDS libraries for GPU acceleration
try:
    import cudf
    import cupy as cp
    import treelite
    from cuml import ForestInference
    print("✅ RAPIDS libraries imported successfully.")
except ImportError:
    print("❌ WARNING: RAPIDS libraries (cudf, cuml, cupy) not found. GPU acceleration will not be available.")
    # Set placeholders if RAPIDS is not installed to avoid script crashing immediately
    cudf = cp = treelite = ForestInference = None

warnings.filterwarnings("ignore")

# --- Configuration ---
INPUT_DIR = "inter"
TRAIN_INPUT_PATH = f"{INPUT_DIR}/train_3_model_ready.parquet"
VALID_INPUT_PATH = f"{INPUT_DIR}/valid_3_model_ready.parquet"
MODEL_DIR = "model"
RANDOM_STATE = 42

# =================================================================================
# === PERFORMANCE OPTIMIZATION: Two-Stage Feature Selection =======================
# =================================================================================
# Stage 1: Use LGBM's fast importance to find the top N candidates.
N_CANDIDATE_FEATURES = 100
# Stage 2: Use the slower, more accurate permutation importance to select the
# final top N features from the candidates.
TOP_N_FEATURES = 50
# =================================================================================


# --- Feature Definitions ---
ALL_CATEGORICAL_FEATURES = [
    "brand_id", "industry_id", "member_industry_code", 
    "offer_start_dayofweek", "is_industry_match", "has_keyword_cashback", 
    "has_keyword_points", "has_keyword_discount", "has_keyword_spend_x", 
    "offer_category", "time_of_day_bin", "offer_lifecycle_stage", 
    "is_weekend", "is_holiday_week", "is_payday_week"
]
TARGET_COL = "y"
GROUP_COLS = ["id2", "id5"]
ID_COLS_TO_EXCLUDE = ["id1", "id3", "id4"]


def optimize_data_types(df):
    """
    Downcasts numerical columns to more memory-efficient types.
    """
    for col in df.select_dtypes('float').columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes('integer').columns:
        # Safely downcast integers, checking min/max ranges
        if df[col].min() >= np.iinfo('int32').min and df[col].max() <= np.iinfo('int32').max:
            df[col] = df[col].astype('int32')
    return df

def load_data(train_path, valid_path):
    """
    Loads and optimizes training and validation data.
    """
    print("--- Loading and optimizing data ---")
    try:
        train_df = pd.read_parquet(train_path, engine='fastparquet')
        valid_df = pd.read_parquet(valid_path, engine='fastparquet')
        
        train_df = optimize_data_types(train_df)
        valid_df = optimize_data_types(valid_df)
        
        print(f"  -> Train data shape: {train_df.shape}")
        print(f"  -> Valid data shape: {valid_df.shape}")
        
        return train_df, valid_df
    except FileNotFoundError as e:
        print(f"❌ ERROR: Could not find input files. Details: {e}")
        return None, None

def calculate_map_at_7_gpu(y_true_gpu, y_pred_gpu, groups_df_gpu, n=7):
    """
    Calculates Mean Average Precision @ 7 entirely on the GPU using cuDF.
    """
    df = groups_df_gpu.copy()
    df['y'] = y_true_gpu
    df['pred'] = cudf.Series(y_pred_gpu)

    map_denominators = df.groupby(GROUP_COLS)['y'].sum().clip(upper=n).rename('map_denominator')

    df = df.sort_values(GROUP_COLS + ["pred"], ascending=[True, True, False])
    df['position'] = df.groupby(GROUP_COLS).cumcount() + 1
    
    df_top7 = df[df['position'] <= n].copy()
    df_top7['cumsum_y'] = df_top7.groupby(GROUP_COLS)['y'].cumsum()
    df_top7['ap_numerator_term'] = (df_top7['cumsum_y'] / df_top7['position']) * df_top7['y']
    ap_numerators = df_top7.groupby(GROUP_COLS)['ap_numerator_term'].sum().rename('map_numerator')
    
    ap_df = ap_numerators.to_frame().merge(
        map_denominators.to_frame(),
        left_index=True,
        right_index=True,
        how='left'
    ).fillna(0)
    
    ap_df['ap_at_7'] = ap_df['map_numerator'] / ap_df['map_denominator']
    
    return ap_df['ap_at_7'].fillna(0).mean()

def train_lgbm_ranker(X_train, y_train, train_groups, X_valid, y_valid, valid_groups, purpose=""):
    """
    Trains an LGBMRanker model.
    """
    print(f"\n--- Training LGBMRanker for {purpose} with {X_train.shape[1]} features ---")
    
    # Ensure categorical features are explicitly converted to 'category' dtype for LightGBM
    categorical_features_for_lgbm = [col for col in X_train.columns if col in ALL_CATEGORICAL_FEATURES]
    for col in categorical_features_for_lgbm:
        if col in X_train.columns:
            X_train[col] = X_train[col].astype('category')
        if col in X_valid.columns:
            X_valid[col] = X_valid[col].astype('category')

    temp_params = {
        'objective': 'lambdarank', 'metric': 'map', 'eval_at': [7],
        'boosting_type': 'gbdt', 'n_estimators': 2000, 'learning_rate': 0.03,
        'num_leaves': 40, 'max_depth': 7, 'colsample_bytree': 0.7, 'subsample': 0.7,
        'seed': RANDOM_STATE, 'feature_fraction_seed': RANDOM_STATE, 'data_random_seed': RANDOM_STATE,
        'n_jobs': -1, 'verbose': -1
    }
    
    model = lgb.LGBMRanker(**temp_params)
    model.fit(X_train, y_train, group=train_groups, eval_set=[(X_valid, y_valid)],
              eval_group=[valid_groups], callbacks=[lgb.early_stopping(100, verbose=False)],
              categorical_feature='auto') # Let LightGBM handle categoricals automatically
    
    return model

def calculate_grouped_permutation_importance(model, X_valid, y_valid, valid_df):
    """
    Calculates group-aware permutation importance on a reduced set of features.
    """
    if cudf is None:
        raise ImportError("RAPIDS must be installed for this function.")

    print("\n--- Preparing data and model for GPU-accelerated Permutation Importance ---")
    model_path = os.path.join(MODEL_DIR, "temp_ranker_for_importance.txt")
    model.booster_.save_model(model_path)
    tl_model = treelite.Model.load(model_path, model_format='lightgbm')
    fil_model = ForestInference.load_from_treelite_model(tl_model)

    feature_cols = X_valid.columns.tolist()

    print("  -> Pre-processing columns by type before GPU conversion...")
    numerical_cols = X_valid.select_dtypes(include=np.number).columns.tolist()
    non_numerical_cols = X_valid.select_dtypes(exclude=np.number).columns.tolist()
    X_valid[numerical_cols] = X_valid[numerical_cols].fillna(0)
    for col_name in non_numerical_cols:
        if not pd.api.types.is_categorical_dtype(X_valid[col_name]):
            X_valid[col_name] = X_valid[col_name].astype('category')
        if '0' not in X_valid[col_name].cat.categories:
            X_valid[col_name] = X_valid[col_name].cat.add_categories(['0'])
        X_valid[col_name] = X_valid[col_name].fillna('0')
    
    X_valid_gpu = cudf.from_pandas(X_valid)
    
    print("  -> Converting categorical columns to numerical codes for prediction...")
    for col_name in non_numerical_cols:
        if col_name in X_valid_gpu.columns:
            X_valid_gpu[col_name] = X_valid_gpu[col_name].cat.codes

    y_valid_gpu = cudf.Series(y_valid.values)
    valid_group_df_gpu = cudf.from_pandas(valid_df[GROUP_COLS])

    print("\n--- Calculating baseline MAP@7 score on GPU ---")
    baseline_preds_gpu = fil_model.predict(X_valid_gpu)
    baseline_score = calculate_map_at_7_gpu(y_valid_gpu, baseline_preds_gpu, valid_group_df_gpu)
    print(f"  -> GPU Baseline MAP@7: {baseline_score:.6f}")

    print(f"\n--- Calculating GROUPED permutation importance on GPU for {len(feature_cols)} features ---")
    importances = {}
    cp.random.seed(RANDOM_STATE)

    shuffle_within_group = lambda x: x.sample(frac=1, random_state=RANDOM_STATE)

    for col in tqdm(feature_cols, desc="Permuting Features on GPU"):
        original_col_gpu = X_valid_gpu[col].copy()
        
        permuted_values = original_col_gpu.groupby(
            [valid_group_df_gpu['id2'], valid_group_df_gpu['id5']]
        ).apply(shuffle_within_group)
        
        X_valid_gpu[col] = permuted_values.values
        
        permuted_preds_gpu = fil_model.predict(X_valid_gpu)
        permuted_score = calculate_map_at_7_gpu(y_valid_gpu, permuted_preds_gpu, valid_group_df_gpu)
        
        X_valid_gpu[col] = original_col_gpu
        importances[col] = baseline_score - permuted_score

    del X_valid_gpu, y_valid_gpu, valid_group_df_gpu, fil_model, tl_model
    if cp is not None:
        cp.get_default_memory_pool().free_all_blocks()
    gc.collect()

    return pd.DataFrame.from_dict(importances, orient="index", columns=["importance"])

def main():
    """
    Main function to orchestrate the two-stage feature selection process.
    """
    print("🚀 Starting Two-Stage GPU-Accelerated Feature Selection")
    os.makedirs(MODEL_DIR, exist_ok=True)

    train_df, valid_df = load_data(TRAIN_INPUT_PATH, VALID_INPUT_PATH)
    if train_df is None:
        return

    base_feature_cols = [col for col in train_df.columns if col not in [TARGET_COL] + GROUP_COLS + ID_COLS_TO_EXCLUDE]
    
    # =========================================================================
    # === CRITICAL CORRECTION: Unify Categorical Feature Definitions ==========
    # =========================================================================
    # The error occurs because LightGBM requires the categories in categorical
    # features to be identical between the training and validation sets.
    # We fix this by explicitly defining the categories from the combined data.
    print("\n--- Unifying categorical feature definitions across train and valid sets ---")
    for col in base_feature_cols:
        if train_df[col].dtype.name in ['category', 'object']:
             # Combine categories from both train and valid sets
            all_categories = pd.concat([train_df[col], valid_df[col]]).astype('category').cat.categories
            # Apply the unified categories to both dataframes
            train_df[col] = pd.Categorical(train_df[col], categories=all_categories)
            valid_df[col] = pd.Categorical(valid_df[col], categories=all_categories)
    # =========================================================================

    # --- Prepare full data sets for initial ranking ---
    X_train_full = train_df[base_feature_cols]
    y_train = train_df[TARGET_COL]
    train_groups = train_df.groupby(GROUP_COLS).size().to_numpy()
    
    X_valid_full = valid_df[base_feature_cols]
    y_valid = valid_df[TARGET_COL]
    valid_groups = valid_df.groupby(GROUP_COLS).size().to_numpy()

    # =========================================================================
    # === STAGE 1: Get Candidate Features from a fast LGBM model ================
    # =========================================================================
    lgbm_full_model = train_lgbm_ranker(
        X_train_full, y_train, train_groups, X_valid_full, y_valid, valid_groups, 
        purpose="Initial Candidate Selection"
    )
    
    importances_df = pd.DataFrame({
        'feature': lgbm_full_model.feature_name_,
        'importance': lgbm_full_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    candidate_features = importances_df.head(N_CANDIDATE_FEATURES)['feature'].tolist()
    print(f"\n  -> Selected top {len(candidate_features)} candidate features from Stage 1.")

    del X_train_full, X_valid_full, lgbm_full_model, importances_df
    gc.collect()

    # =========================================================================
    # === STAGE 2: Run Permutation Importance on Candidate Features ===========
    # =========================================================================
    # Filter data to only include the promising candidates
    X_train_cand = train_df[candidate_features]
    X_valid_cand = valid_df[candidate_features]

    # Train a new model specifically for the permutation importance step
    pi_model = train_lgbm_ranker(
        X_train_cand, y_train, train_groups, X_valid_cand, y_valid, valid_groups,
        purpose="Permutation Importance"
    )

    # Calculate final importances using the more accurate (but slower) method
    final_importance_df = calculate_grouped_permutation_importance(pi_model, X_valid_cand, y_valid, valid_df)
    
    # --- Process and save results ---
    if not final_importance_df.empty:
        final_importance_df.sort_values(by="importance", ascending=False, inplace=True)
        
        full_importance_path = os.path.join(MODEL_DIR, "full_permutation_importances_gpu.csv")
        final_importance_df.to_csv(full_importance_path)
        print(f"\n✅ Full permutation importance scores saved to '{full_importance_path}'")

        positive_importance_df = final_importance_df[final_importance_df["importance"] > 0]
        top_features = positive_importance_df.head(TOP_N_FEATURES).index.tolist()

        print(f"\n--- Selected final top {len(top_features)} features from Permutation Importance ---")
        print("Top 15 features:")
        print(positive_importance_df.head(15))

        selected_features_path = os.path.join(MODEL_DIR, f"top_{TOP_N_FEATURES}_features_gpu.json")
        with open(selected_features_path, "w") as f:
            json.dump(top_features, f, indent=4)
        print(f"\n✅ Top feature list saved to '{selected_features_path}'")

    print("\n--- Feature Selection Script Finished ---")


if __name__ == "__main__":
    if cudf is None:
        print("❌ This script requires RAPIDS to run. Please install cudf, cuml, and cupy.")
    else:
        main()


In [1]:
import pandas as pd
import numpy as np
import os
import gc
import json
import warnings
from tqdm import tqdm
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# --- Configuration ---
INPUT_DIR = "inter"
TRAIN_INPUT_PATH = f"{INPUT_DIR}/train_3_model_ready.parquet"
MODEL_DIR = "model"
TOP_FEATURES_PATH = f"{MODEL_DIR}/top_50_features_gpu.json"
DATA_DICTIONARY_PATH = "data_dictionary.csv" # Path to your data dictionary

# --- Optuna Settings ---
N_TRIALS = 50
TUNING_VALIDATION_RATIO = 0.2
RANDOM_STATE = 42

def calculate_map_at_7(y_true, y_pred, groups_df):
    """
    Calculates Mean Average Precision @ 7, based on the competition's logic.
    """
    df = pd.DataFrame({"y": y_true.values, "pred": y_pred})
    df = pd.concat([groups_df.reset_index(drop=True), df], axis=1)
    
    # =========================================================================
    # === CRITICAL CORRECTION: Fix sorting for ranking metric calculation =====
    # =========================================================================
    # The groups (id2, id5) must be sorted ascending to keep them together,
    # while the predictions (pred) must be sorted descending to rank them.
    df = df.sort_values(["id2", "id5", "pred"], ascending=[True, True, False]).reset_index(drop=True)
    # =========================================================================

    df["cumsum_y"] = df.groupby(["id2", "id5"])["y"].cumsum()
    df["position"] = df.groupby(["id2", "id5"]).cumcount() + 1
    df["ap_numerator_term"] = (df["cumsum_y"] / df["position"]) * df["y"]

    ap_df = df.groupby(["id2", "id5"]).agg(
        map_numerator=("ap_numerator_term", "sum"),
        map_denominator=("y", lambda x: min(x.sum(), 7)),
    )
    ap_df["ap_at_7"] = ap_df.apply(
        lambda row: row["map_numerator"] / row["map_denominator"]
        if row["map_denominator"] > 0
        else 0,
        axis=1,
    )
    return ap_df["ap_at_7"].mean()


def objective(
    trial, tune_train_df, tune_valid_df, feature_cols
):
    """
    The objective function that Optuna will try to maximize.
    """
    params = {
        "objective": "lambdarank",
        "metric": "map",
        "eval_at": [7],
        "boosting_type": "gbdt",
        "seed": RANDOM_STATE,
        "n_jobs": -1,
        "verbose": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 80),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
    }

    X_train, y_train = tune_train_df[feature_cols], tune_train_df["y"]
    X_valid, y_valid = tune_valid_df[feature_cols], tune_valid_df["y"]

    train_groups = tune_train_df.groupby(["id2", "id5"]).size().to_numpy()
    valid_groups = tune_valid_df.groupby(["id2", "id5"]).size().to_numpy()

    pruning_callback = LightGBMPruningCallback(trial, "map@7")

    model = lgb.LGBMRanker(**params, n_estimators=1000)
    model.fit(
        X_train,
        y_train,
        group=train_groups,
        eval_set=[(X_valid, y_valid)],
        eval_group=[valid_groups],
        callbacks=[lgb.early_stopping(50, verbose=False), pruning_callback],
        categorical_feature='auto', # Let LightGBM handle columns with 'category' dtype
    )

    preds = model.predict(X_valid)
    valid_groups_df = tune_valid_df[["id2", "id5"]]
    score = calculate_map_at_7(y_valid, preds, valid_groups_df)

    return score


if __name__ == "__main__":
    print("🚀 Starting Hyperparameter Tuning for LGBMRanker with Optuna (Revised)")
    os.makedirs(MODEL_DIR, exist_ok=True)

    print("\n--- Step 1: Loading data and top features ---")
    try:
        train_df = pd.read_parquet(TRAIN_INPUT_PATH)
        with open(TOP_FEATURES_PATH, "r") as f:
            top_features = json.load(f)
        # Load the data dictionary to define data types
        data_dict = pd.read_csv(DATA_DICTIONARY_PATH)
        print(f"  -> Loaded training data with shape: {train_df.shape}")
        print(f"  -> Loaded {len(top_features)} features from '{TOP_FEATURES_PATH}'")
        print(f"  -> Loaded data dictionary from '{DATA_DICTIONARY_PATH}'")
    except FileNotFoundError as e:
        print(
            f"❌ ERROR: Could not find a required file. Please run previous steps. Details: {e}"
        )
        exit()

    print("\n--- Step 2: Creating time-based tuning split ---")
    # The data should already be sorted by time from previous steps, but we ensure it here.
    train_df["id5"] = pd.to_datetime(train_df["id5"])
    train_df = train_df.sort_values("id5").reset_index(drop=True)

    split_point = int(len(train_df) * (1 - TUNING_VALIDATION_RATIO))
    tune_train_df = train_df.iloc[:split_point]
    tune_valid_df = train_df.iloc[split_point:]

    print(f"  -> Tuning train shape: {tune_train_df.shape}")
    print(f"  -> Tuning valid shape: {tune_valid_df.shape}")
    del train_df
    gc.collect()

    # =========================================================================
    # === CRITICAL CORRECTION: Pre-process dtypes using a Robust Strategy ====
    # =========================================================================
    # Use the data dictionary and dtype inspection to build a comprehensive list
    # of all non-numerical features, ensuring none are missed.
    print("\n--- Pre-processing data types for LightGBM using Data Dictionary and dtype inspection ---")
    
    # 1. Get categorical features from the data dictionary
    dict_cat_cols = data_dict[data_dict['Type'] == 'Categorical']['masked_column'].tolist()
    
    # 2. Get any other non-numerical features directly from the DataFrame dtypes
    # This catches any feature-engineered columns that might not be in the dictionary.
    other_obj_cols = tune_train_df[top_features].select_dtypes(include=['object', 'category']).columns.tolist()

    # 3. Combine them into a single, unique list
    categorical_features_to_convert = sorted(list(set(dict_cat_cols + other_obj_cols)))
    
    # 4. Filter this list to only include features present in our `top_features` list
    final_cat_cols = [col for col in categorical_features_to_convert if col in top_features]

    print(f"  -> Identified {len(final_cat_cols)} non-numerical features to process.")

    for col in final_cat_cols:
        # Unify categories between train and valid sets to prevent errors
        all_categories = pd.concat([tune_train_df[col], tune_valid_df[col]]).astype('category').cat.categories
        tune_train_df[col] = pd.Categorical(tune_train_df[col], categories=all_categories)
        tune_valid_df[col] = pd.Categorical(tune_valid_df[col], categories=all_categories)
    
    print(f"  -> Successfully converted features to 'category' dtype.")
    # =========================================================================

    print(f"\n--- Step 3: Starting Optuna study with {N_TRIALS} trials ---")
    study = optuna.create_study(direction="maximize", study_name="lgbm_ranker_tuning")

    study.optimize(
        lambda trial: objective(
            trial,
            tune_train_df,
            tune_valid_df,
            top_features,
        ),
        n_trials=N_TRIALS,
        show_progress_bar=True,
    )

    best_params = study.best_params
    print("\n--- Study Complete ---")
    print(f"  -> Best MAP@7 score: {study.best_value:.6f}")
    print("  -> Best hyperparameters found:")
    for key, value in best_params.items():
        print(f"       - {key}: {value}")

    best_params.update(
        {
            "objective": "lambdarank",
            "metric": "map",
            "eval_at": [7],
            "boosting_type": "gbdt",
            "seed": RANDOM_STATE,
            "n_jobs": -1,
            "verbose": -1,
        }
    )

    params_path = os.path.join(MODEL_DIR, "best_lgbm_params.json")
    with open(params_path, "w") as f:
        json.dump(best_params, f, indent=4)

    print(f"\n✅ Best parameters saved to '{params_path}'")


🚀 Starting Hyperparameter Tuning for LGBMRanker with Optuna (Revised)

--- Step 1: Loading data and top features ---
  -> Loaded training data with shape: (589815, 3097)
  -> Loaded 50 features from 'model/top_50_features_gpu.json'
  -> Loaded data dictionary from 'data_dictionary.csv'

--- Step 2: Creating time-based tuning split ---
  -> Tuning train shape: (471852, 3097)
  -> Tuning valid shape: (117963, 3097)

--- Pre-processing data types for LightGBM using Data Dictionary and dtype inspection ---
  -> Identified 34 non-numerical features to process.


[I 2025-07-27 19:05:51,251] A new study created in memory with name: lgbm_ranker_tuning


  -> Successfully converted features to 'category' dtype.

--- Step 3: Starting Optuna study with 50 trials ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-07-27 19:06:18,761] Trial 0 finished with value: 0.08036400292848411 and parameters: {'learning_rate': 0.019264162882174963, 'num_leaves': 30, 'max_depth': 10, 'subsample': 0.835030001627035, 'colsample_bytree': 0.6693605035581052, 'reg_alpha': 0.19318856520061312, 'reg_lambda': 0.004839661170136858}. Best is trial 0 with value: 0.08036400292848411.
[I 2025-07-27 19:06:24,926] Trial 1 finished with value: 0.08078165985578585 and parameters: {'learning_rate': 0.03380356188343687, 'num_leaves': 44, 'max_depth': 9, 'subsample': 0.9186289871663155, 'colsample_bytree': 0.8183633354819033, 'reg_alpha': 0.14916682279615115, 'reg_lambda': 5.348000217056512}. Best is trial 1 with value: 0.08078165985578585.
[I 2025-07-27 19:06:36,815] Trial 2 finished with value: 0.07989251775698254 and parameters: {'learning_rate': 0.010706434693760253, 'num_leaves': 50, 'max_depth': 6, 'subsample': 0.8297891152299803, 'colsample_bytree': 0.8410738827348762, 'reg_alpha': 9.906569505157302, 'reg_lambda'

In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedGroupKFold
import os
import gc
import json
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# --- Configuration ---
INPUT_DIR = "inter"
TRAIN_INPUT_PATH = f"{INPUT_DIR}/train_3_model_ready.parquet"
VALID_INPUT_PATH = f"{INPUT_DIR}/valid_3_model_ready.parquet"
TEST_INPUT_PATH = f"{INPUT_DIR}/test_3_model_ready.parquet"
DATA_DICTIONARY_PATH = "data_dictionary.csv" # Path to your data dictionary

MODEL_DIR = "model"
OUTPUT_DIR = "inter"

# Paths to the artifacts created by previous scripts
TOP_FEATURES_PATH = f"{MODEL_DIR}/top_50_features_gpu.json"
BEST_PARAMS_PATH = f"{MODEL_DIR}/best_lgbm_params.json"

N_SPLITS = 5
RANDOM_STATE = 42

# ==============================================================================
# === MAIN EXECUTION ===========================================================
# ==============================================================================

if __name__ == "__main__":
    print("🚀 Starting LGBM Out-of-Fold (OOF) Prediction Generation (Revised)")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(MODEL_DIR, exist_ok=True)

    # --- Step 1: Load Data and Modeling Artifacts ---
    print("\n--- Step 1: Loading data and artifacts ---")
    try:
        train_df = pd.read_parquet(TRAIN_INPUT_PATH)
        valid_df = pd.read_parquet(VALID_INPUT_PATH)
        test_df = pd.read_parquet(TEST_INPUT_PATH)

        with open(TOP_FEATURES_PATH, "r") as f:
            top_features = json.load(f)

        with open(BEST_PARAMS_PATH, "r") as f:
            lgbm_params = json.load(f)
            
        data_dict = pd.read_csv(DATA_DICTIONARY_PATH)

        print(f"  -> Loaded training data with shape: {train_df.shape}")
        print(f"  -> Loaded {len(top_features)} features from '{TOP_FEATURES_PATH}'")
        print(f"  -> Loaded best hyperparameters from '{BEST_PARAMS_PATH}'")
        print(f"  -> Loaded data dictionary from '{DATA_DICTIONARY_PATH}'")

    except FileNotFoundError as e:
        print(f"❌ ERROR: Could not find a required file. Please run all previous steps. Details: {e}")
        exit()

    # =========================================================================
    # === CRITICAL CORRECTION: Pre-process dtypes using a Robust Strategy ====
    # =========================================================================
    # Use the data dictionary and dtype inspection to build a comprehensive list
    # of all non-numerical features, ensuring none are missed.
    print("\n--- Pre-processing data types for LightGBM using Data Dictionary and dtype inspection ---")
    
    # 1. Get categorical features from the data dictionary
    dict_cat_cols = data_dict[data_dict['Type'] == 'Categorical']['masked_column'].tolist()
    
    # 2. Get any other non-numerical features directly from the DataFrame dtypes
    other_obj_cols = train_df[top_features].select_dtypes(include=['object', 'category']).columns.tolist()

    # 3. Combine them into a single, unique list
    categorical_features_to_convert = sorted(list(set(dict_cat_cols + other_obj_cols)))
    
    # 4. Filter this list to only include features present in our `top_features` list
    final_cat_cols = [col for col in categorical_features_to_convert if col in top_features]

    print(f"  -> Identified {len(final_cat_cols)} non-numerical features to process.")

    for col in final_cat_cols:
        # Unify categories across all data splits to prevent errors
        all_categories = pd.concat([train_df[col], valid_df[col], test_df[col]]).astype('category').cat.categories
        train_df[col] = pd.Categorical(train_df[col], categories=all_categories)
        valid_df[col] = pd.Categorical(valid_df[col], categories=all_categories)
        test_df[col] = pd.Categorical(test_df[col], categories=all_categories)
    
    print(f"  -> Successfully converted features to 'category' dtype across all data splits.")
    # =========================================================================

    # --- Step 2: Prepare DataFrames for Ranking ---
    print("\n--- Step 2: Preparing data with selected features ---")
    # Sort all dataframes by customer and time to correctly calculate group sizes later
    train_df = train_df.sort_values(by=["id2", "id5", "id4"]).reset_index(drop=True)
    valid_df = valid_df.sort_values(by=["id2", "id5", "id4"]).reset_index(drop=True)
    test_df = test_df.sort_values(by=["id2", "id5", "id4"]).reset_index(drop=True)

    target_col = "y"
    X_train = train_df[top_features]
    y_train = train_df[target_col]
    groups_for_split = train_df["id2"]

    X_valid = valid_df[top_features]
    X_test = test_df[top_features]

    # --- Step 3: OOF Generation with StratifiedGroupKFold ---
    print("\n--- Step 3: Generating OOF predictions with LGBMRanker ---")
    sgkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

    oof_preds = np.zeros(len(train_df))
    valid_preds_ensemble = np.zeros(len(valid_df))
    test_preds_ensemble = np.zeros(len(test_df))

    for fold, (train_idx, val_idx) in enumerate(
        sgkf.split(X_train, y_train, groups=groups_for_split)
    ):
        print(f"\n  -> Processing Fold {fold + 1}/{N_SPLITS}...")
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]

        train_fold_groups = train_df.iloc[train_idx].groupby(["id2", "id5"]).size().to_numpy()
        val_fold_groups = train_df.iloc[val_idx].groupby(["id2", "id5"]).size().to_numpy()

        model = lgb.LGBMRanker(**lgbm_params, n_estimators=2000)
        model.fit(
            X_train_fold,
            y_train_fold,
            group=train_fold_groups,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_group=[val_fold_groups],
            callbacks=[lgb.early_stopping(100, verbose=False)],
            categorical_feature='auto', # Let LightGBM handle categoricals automatically
        )

        model_path = os.path.join(MODEL_DIR, f"lgbm_ranker_fold_{fold + 1}.txt")
        model.booster_.save_model(model_path)
        print(f"  -> Model for fold {fold + 1} saved to '{model_path}'")

        oof_preds[val_idx] = model.predict(X_val_fold)
        valid_preds_ensemble += model.predict(X_valid) / N_SPLITS
        test_preds_ensemble += model.predict(X_test) / N_SPLITS

    train_df["oof_lgbm_prediction"] = oof_preds
    valid_df["oof_lgbm_prediction"] = valid_preds_ensemble
    test_df["oof_lgbm_prediction"] = test_preds_ensemble
    print("\n  -> OOF and Ensemble predictions created successfully.")

    # --- Step 4: Save the final datasets ---
    print("\n--- Step 4: Saving new datasets with OOF features ---")
    train_df.to_parquet(f"{OUTPUT_DIR}/train_4_oof_lgbm.parquet", index=False)
    valid_df.to_parquet(f"{OUTPUT_DIR}/valid_4_oof_lgbm.parquet", index=False)
    test_df.to_parquet(f"{OUTPUT_DIR}/test_4_oof_lgbm.parquet", index=False)

    print(f"  -> Saved 'train_4_oof_lgbm.parquet' with shape {train_df.shape}")
    print(f"  -> Saved 'valid_4_oof_lgbm.parquet' with shape {valid_df.shape}")
    print(f"  -> Saved 'test_4_oof_lgbm.parquet' with shape {test_df.shape}")

    del train_df, valid_df, test_df, X_train, y_train, X_valid, X_test
    gc.collect()

    print("\n✅ --- LGBM OOF Feature Generation Complete --- ✅")


🚀 Starting LGBM Out-of-Fold (OOF) Prediction Generation (Revised)

--- Step 1: Loading data and artifacts ---
  -> Loaded training data with shape: (589815, 3097)
  -> Loaded 50 features from 'model/top_50_features_gpu.json'
  -> Loaded best hyperparameters from 'model/best_lgbm_params.json'
  -> Loaded data dictionary from 'data_dictionary.csv'

--- Pre-processing data types for LightGBM using Data Dictionary and dtype inspection ---
  -> Identified 34 non-numerical features to process.
  -> Successfully converted features to 'category' dtype across all data splits.

--- Step 2: Preparing data with selected features ---

--- Step 3: Generating OOF predictions with LGBMRanker ---

  -> Processing Fold 1/5...
  -> Model for fold 1 saved to 'model/lgbm_ranker_fold_1.txt'

  -> Processing Fold 2/5...
  -> Model for fold 2 saved to 'model/lgbm_ranker_fold_2.txt'

  -> Processing Fold 3/5...
  -> Model for fold 3 saved to 'model/lgbm_ranker_fold_3.txt'

  -> Processing Fold 4/5...
  -> Mode

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import json
import warnings
import xgboost as xgb
import optuna
from optuna.integration import XGBoostPruningCallback

# --- Suppress Warnings for a Cleaner Output ---
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# --- Configuration ---
INPUT_DIR = "inter"
TRAIN_INPUT_PATH = f"{INPUT_DIR}/train_3_model_ready.parquet"
MODEL_DIR = "model"
TOP_FEATURES_PATH = f"{MODEL_DIR}/top_50_features_gpu.json"
DATA_DICTIONARY_PATH = "data_dictionary.csv"

# --- Optuna Settings ---
N_TRIALS = 20
TUNING_VALIDATION_RATIO = 0.2
RANDOM_STATE = 42


def calculate_map_at_7(y_true, y_pred, groups_df):
    """
    Calculates Mean Average Precision @ 7.
    """
    df = pd.DataFrame({"y": y_true, "pred": y_pred})
    df = pd.concat([groups_df.reset_index(drop=True), df], axis=1)
    df = df.sort_values(["id2", "id5", "pred"], ascending=[True, True, False]).reset_index(drop=True)
    df["cumsum_y"] = df.groupby(["id2", "id5"])["y"].cumsum()
    df["position"] = df.groupby(["id2", "id5"]).cumcount() + 1
    df["ap_numerator_term"] = (df["cumsum_y"] / df["position"]) * df["y"]
    ap_df = df.groupby(["id2", "id5"]).agg(
        map_numerator=("ap_numerator_term", "sum"),
        map_denominator=("y", lambda x: min(x.sum(), 7)),
    )
    ap_df["ap_at_7"] = ap_df.apply(
        lambda row: row["map_numerator"] / row["map_denominator"]
        if row["map_denominator"] > 0
        else 0,
        axis=1,
    )
    return ap_df["ap_at_7"].mean()


def objective(trial, tune_train_df, tune_valid_df, feature_cols, qid_train, qid_valid):
    """
    The objective function for XGBoost that Optuna will try to maximize.
    """
    params = {
        "objective": "rank:map",
        "eval_metric": "map@7",
        "booster": "gbtree",
        "seed": RANDOM_STATE,
        "n_jobs": -1,
        "enable_categorical": True,
        "tree_method": "hist",
        "eta": trial.suggest_float("eta", 0.01, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
    }

    X_train, y_train = tune_train_df[feature_cols], tune_train_df["y"]
    X_valid, y_valid = tune_valid_df[feature_cols], tune_valid_df["y"]

    pruning_callback = XGBoostPruningCallback(trial, "validation_0-map@7")

    model = xgb.XGBRanker(
        **params, 
        n_estimators=1000,
        callbacks=[pruning_callback]
    )

    model.fit(
        X_train,
        y_train,
        qid=qid_train,
        eval_set=[(X_valid, y_valid)],
        eval_qid=[qid_valid],
        verbose=False,
    )

    preds = model.predict(X_valid)
    valid_groups_df = tune_valid_df[["id2", "id5"]]
    score = calculate_map_at_7(y_valid, preds, valid_groups_df)

    return score


if __name__ == "__main__":
    print("🚀 Starting Hyperparameter Tuning for XGBRanker with Optuna")
    os.makedirs(MODEL_DIR, exist_ok=True)

    print("\n--- Step 1: Loading data and top features ---")
    try:
        train_df = pd.read_parquet(TRAIN_INPUT_PATH)
        with open(TOP_FEATURES_PATH, "r") as f:
            top_features = json.load(f)
        data_dict = pd.read_csv(DATA_DICTIONARY_PATH)
        print(f"  -> Loaded training data with shape: {train_df.shape}")
        print(f"  -> Loaded {len(top_features)} features from '{TOP_FEATURES_PATH}'")
        print(f"  -> Loaded data dictionary from '{DATA_DICTIONARY_PATH}'")
    except FileNotFoundError as e:
        print(f"❌ ERROR: Could not find a required file. Please run previous steps. Details: {e}")
        exit()

    print("\n--- Step 2: Creating time-based tuning split ---")
    train_df["id5"] = pd.to_datetime(train_df["id5"])
    train_df = train_df.sort_values("id5").reset_index(drop=True)
    split_point = int(len(train_df) * (1 - TUNING_VALIDATION_RATIO))
    tune_train_df = train_df.iloc[:split_point].copy()
    tune_valid_df = train_df.iloc[split_point:].copy()
    print(f"  -> Tuning train shape: {tune_train_df.shape}")
    print(f"  -> Tuning valid shape: {tune_valid_df.shape}")
    del train_df
    gc.collect()

    print("\n--- Step 3: Preparing data for XGBoost ---")
    print("  -> Identifying non-numerical features to convert...")
    dict_cat_cols = data_dict[data_dict['Type'] == 'Categorical']['masked_column'].tolist()
    other_obj_cols = tune_train_df[top_features].select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_features_to_convert = sorted(list(set(dict_cat_cols + other_obj_cols)))
    final_cat_cols = [col for col in categorical_features_to_convert if col in top_features]
    print(f"  -> Found {len(final_cat_cols)} features to convert to 'category' dtype.")

    for col in final_cat_cols:
        all_categories = pd.concat([tune_train_df[col], tune_valid_df[col]]).astype('category').cat.categories
        tune_train_df[col] = pd.Categorical(tune_train_df[col], categories=all_categories)
        tune_valid_df[col] = pd.Categorical(tune_valid_df[col], categories=all_categories)
    print("  -> Successfully converted features and unified categories.")
    
    # =========================================================================
    # === ✨ CRITICAL FIX: Sort dataframes to group queries together ✨ ===
    # =========================================================================
    print("  -> Sorting dataframes by query ID to meet XGBoost requirements...")
    tune_train_df = tune_train_df.sort_values(["id2", "id5"]).reset_index(drop=True)
    tune_valid_df = tune_valid_df.sort_values(["id2", "id5"]).reset_index(drop=True)
    # =========================================================================

    print("  -> Creating Query IDs (qid) for XGBoost...")
    qid_train = pd.factorize(tune_train_df["id2"].astype(str) + "_" + tune_train_df["id5"].astype(str))[0]
    qid_valid = pd.factorize(tune_valid_df["id2"].astype(str) + "_" + tune_valid_df["id5"].astype(str))[0]

    print(f"\n--- Step 4: Starting Optuna study with {N_TRIALS} trials ---")
    study = optuna.create_study(direction="maximize", study_name="xgb_ranker_tuning")

    study.optimize(
        lambda trial: objective(
            trial, tune_train_df, tune_valid_df, top_features, qid_train, qid_valid
        ),
        n_trials=N_TRIALS,
        show_progress_bar=True,
    )

    best_params = study.best_params
    print("\n--- Study Complete ---")
    print(f"  -> Best MAP@7 score: {study.best_value:.6f}")
    print("  -> Best hyperparameters found:")
    for key, value in best_params.items():
        print(f"        - {key}: {value}")

    best_params.update(
        {
            "objective": "rank:map",
            "eval_metric": "map@7",
            "booster": "gbtree",
            "seed": RANDOM_STATE,
            "n_jobs": -1,
            "enable_categorical": True,
            "tree_method": "hist",
        }
    )

    params_path = os.path.join(MODEL_DIR, "best_xgb_params.json")
    with open(params_path, "w") as f:
        json.dump(best_params, f, indent=4)

    print(f"\n✅ Best XGBoost parameters saved to '{params_path}'")



🚀 Starting Hyperparameter Tuning for XGBRanker with Optuna

--- Step 1: Loading data and top features ---
  -> Loaded training data with shape: (589815, 3097)
  -> Loaded 50 features from 'model/top_50_features_gpu.json'
  -> Loaded data dictionary from 'data_dictionary.csv'

--- Step 2: Creating time-based tuning split ---
  -> Tuning train shape: (471852, 3097)
  -> Tuning valid shape: (117963, 3097)

--- Step 3: Preparing data for XGBoost ---
  -> Identifying non-numerical features to convert...
  -> Found 34 features to convert to 'category' dtype.
  -> Successfully converted features and unified categories.
  -> Sorting dataframes by query ID to meet XGBoost requirements...
  -> Creating Query IDs (qid) for XGBoost...


[I 2025-07-27 19:04:19,819] A new study created in memory with name: xgb_ranker_tuning



--- Step 4: Starting Optuna study with 20 trials ---


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedGroupKFold
import os
import gc
import json
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# --- Configuration ---
INPUT_DIR = "inter"
# NOTE: Using the output from the LGBM OOF script as input
TRAIN_INPUT_PATH = f"{INPUT_DIR}/train_4_oof_lgbm.parquet"
VALID_INPUT_PATH = f"{INPUT_DIR}/valid_4_oof_lgbm.parquet"
TEST_INPUT_PATH = f"{INPUT_DIR}/test_4_oof_lgbm.parquet"
DATA_DICTIONARY_PATH = "data_dictionary.csv" # Path to your data dictionary

MODEL_DIR = "model"
OUTPUT_DIR = "inter"

# Paths to the artifacts created by previous scripts
TOP_FEATURES_PATH = f"{MODEL_DIR}/top_50_features_gpu.json"
BEST_PARAMS_PATH = f"{MODEL_DIR}/best_xgb_params.json"

N_SPLITS = 5
RANDOM_STATE = 42

# ==============================================================================
# === MAIN EXECUTION ===========================================================
# ==============================================================================

if __name__ == "__main__":
    print("🚀 Starting XGBoost Out-of-Fold (OOF) Prediction Generation")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(MODEL_DIR, exist_ok=True)

    # --- Step 1: Load Data and Modeling Artifacts ---
    print("\n--- Step 1: Loading data and artifacts ---")
    try:
        train_df = pd.read_parquet(TRAIN_INPUT_PATH)
        valid_df = pd.read_parquet(VALID_INPUT_PATH)
        test_df = pd.read_parquet(TEST_INPUT_PATH)

        with open(TOP_FEATURES_PATH, "r") as f:
            top_features = json.load(f)

        with open(BEST_PARAMS_PATH, "r") as f:
            xgb_params = json.load(f)
            
        data_dict = pd.read_csv(DATA_DICTIONARY_PATH)

        print(f"  -> Loaded training data with shape: {train_df.shape}")
        print(f"  -> Loaded {len(top_features)} features from '{TOP_FEATURES_PATH}'")
        print(f"  -> Loaded best XGBoost hyperparameters from '{BEST_PARAMS_PATH}'")
        print(f"  -> Loaded data dictionary from '{DATA_DICTIONARY_PATH}'")

    except FileNotFoundError as e:
        print(
            f"❌ ERROR: Could not find a required file. Please run all previous steps. Details: {e}"
        )
        exit()

    # =========================================================================
    # === CRITICAL CORRECTION: Pre-process dtypes using a Robust Strategy ====
    # =========================================================================
    # Use the data dictionary and dtype inspection to build a comprehensive list
    # of all non-numerical features, ensuring none are missed.
    print("\n--- Pre-processing data types for XGBoost using Data Dictionary and dtype inspection ---")
    
    # 1. Get categorical features from the data dictionary
    dict_cat_cols = data_dict[data_dict['Type'] == 'Categorical']['masked_column'].tolist()
    
    # 2. Get any other non-numerical features directly from the DataFrame dtypes
    other_obj_cols = train_df[top_features].select_dtypes(include=['object', 'category']).columns.tolist()

    # 3. Combine them into a single, unique list
    categorical_features_to_convert = sorted(list(set(dict_cat_cols + other_obj_cols)))
    
    # 4. Filter this list to only include features present in our `top_features` list
    final_cat_cols = [col for col in categorical_features_to_convert if col in top_features]

    print(f"  -> Identified {len(final_cat_cols)} non-numerical features to process.")

    for col in final_cat_cols:
        # Unify categories across all data splits to prevent errors
        all_categories = pd.concat([train_df[col], valid_df[col], test_df[col]]).astype('category').cat.categories
        train_df[col] = pd.Categorical(train_df[col], categories=all_categories)
        valid_df[col] = pd.Categorical(valid_df[col], categories=all_categories)
        test_df[col] = pd.Categorical(test_df[col], categories=all_categories)
    
    print(f"  -> Successfully converted features to 'category' dtype across all data splits.")
    # =========================================================================

    # --- Step 2: Prepare DataFrames for XGBoost Ranking ---
    print("\n--- Step 2: Preparing data with selected features ---")
    train_df = train_df.sort_values(by=["id2", "id5", "id4"]).reset_index(drop=True)
    valid_df = valid_df.sort_values(by=["id2", "id5", "id4"]).reset_index(drop=True)
    test_df = test_df.sort_values(by=["id2", "id5", "id4"]).reset_index(drop=True)

    # Add the LGBM OOF prediction as a feature
    all_features = top_features + ["oof_lgbm_prediction"]

    target_col = "y"
    X_train = train_df[all_features]
    y_train = train_df[target_col]
    groups_for_split = train_df["id2"]

    X_valid = valid_df[all_features]
    X_test = test_df[all_features]

    # --- Step 3: OOF Generation with StratifiedGroupKFold ---
    print("\n--- Step 3: Generating OOF predictions with XGBRanker ---")
    sgkf = StratifiedGroupKFold(
        n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE
    )

    oof_preds = np.zeros(len(train_df))
    valid_preds_ensemble = np.zeros(len(valid_df))
    test_preds_ensemble = np.zeros(len(test_df))

    for fold, (train_idx, val_idx) in enumerate(
        sgkf.split(X_train, y_train, groups=groups_for_split)
    ):
        print(f"\n  -> Processing Fold {fold + 1}/{N_SPLITS}...")
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]

        # Create Query IDs (qid) for the current fold
        qid_train_fold = pd.factorize(
            train_df.iloc[train_idx]["id2"].astype(str)
            + "_"
            + train_df.iloc[train_idx]["id5"].astype(str)
        )[0]
        qid_val_fold = pd.factorize(
            train_df.iloc[val_idx]["id2"].astype(str)
            + "_"
            + train_df.iloc[val_idx]["id5"].astype(str)
        )[0]
        
        # Create qids for the full validation and test sets for prediction
        qid_valid_full = pd.factorize(
            valid_df["id2"].astype(str) + "_" + valid_df["id5"].astype(str)
        )[0]
        qid_test_full = pd.factorize(
            test_df["id2"].astype(str) + "_" + test_df["id5"].astype(str)
        )[0]


        model = xgb.XGBRanker(**xgb_params, n_estimators=2000)
        model.fit(
            X_train_fold,
            y_train_fold,
            qid=qid_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_qid=[qid_val_fold],
            callbacks=[xgb.callback.EarlyStopping(rounds=100, save_best=True)],
            verbose=False,
        )

        model_path = os.path.join(MODEL_DIR, f"xgb_ranker_fold_{fold + 1}.json")
        model.save_model(model_path)
        print(f"  -> Model for fold {fold + 1} saved to '{model_path}'")

        oof_preds[val_idx] = model.predict(X_val_fold)
        valid_preds_ensemble += model.predict(X_valid) / N_SPLITS
        test_preds_ensemble += model.predict(X_test) / N_SPLITS

    train_df["oof_xgb_prediction"] = oof_preds
    valid_df["oof_xgb_prediction"] = valid_preds_ensemble
    test_df["oof_xgb_prediction"] = test_preds_ensemble
    print("\n  -> OOF and Ensemble predictions created successfully.")

    # --- Step 4: Save the final datasets ---
    print("\n--- Step 4: Saving new datasets with OOF features ---")
    train_df.to_parquet(f"{OUTPUT_DIR}/train_5_oof_xgb.parquet", index=False)
    valid_df.to_parquet(f"{OUTPUT_DIR}/valid_5_oof_xgb.parquet", index=False)
    test_df.to_parquet(f"{OUTPUT_DIR}/test_5_oof_xgb.parquet", index=False)

    print(f"  -> Saved 'train_5_oof_xgb.parquet' with shape {train_df.shape}")
    print(f"  -> Saved 'valid_5_oof_xgb.parquet' with shape {valid_df.shape}")
    print(f"  -> Saved 'test_5_oof_xgb.parquet' with shape {test_df.shape}")

    del train_df, valid_df, test_df, X_train, y_train, X_valid, X_test
    gc.collect()

    print("\n✅ --- XGBoost OOF Feature Generation Complete --- ✅")


In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import warnings

# --- NEW DEPENDENCY: torch_scatter ---
# This library is required for the vectorized listwise loss function.
# Please install it using: pip install torch-scatter
try:
    from torch_scatter import scatter_softmax, scatter_sum
except ImportError:
    print("❌ WARNING: torch_scatter is not installed. The vectorized loss function will not work.")
    print("Please install it: pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu121.html (adjust for your torch/cuda version)")
    # Define dummy functions if not installed to prevent immediate script crash
    def scatter_softmax(src, index, dim): return src
    def scatter_sum(src, index, dim): return src.sum()


# =================================================================================
# 🚀 SCRIPT 2: RESIDUAL TRANSFORMER (WITH VECTORIZED LISTWISE RANKING LOSS)
# =================================================================================

# --- Competition Metric Calculation (Verified Correct) ---
def calculate_map_at_7(y_true, y_pred, groups_df):
    """
    Calculates Mean Average Precision @ 7, with the corrected denominator
    to match the official competition scoring logic.
    """
    df = pd.DataFrame({"y": y_true, "pred": y_pred})
    df = pd.concat([groups_df.reset_index(drop=True), df], axis=1)
    df = df.sort_values(["id2", "id5", "pred"], ascending=[True, True, False])
    df["position"] = df.groupby(["id2", "id5"]).cumcount() + 1
    df_top7 = df[df['position'] <= 7].copy()
    df_top7['cumsum_y'] = df_top7.groupby(["id2", "id5"])['y'].cumsum()
    df_top7['ap_numerator_term'] = (df_top7['cumsum_y'] / df_top7['position']) * df_top7['y']
    ap_numerators = df_top7.groupby(["id2", "id5"])['ap_numerator_term'].sum()
    ap_denominators = df.groupby(["id2", "id5"])['y'].sum().clip(upper=7)
    ap_df = pd.concat([ap_numerators, ap_denominators], axis=1).fillna(0)
    ap_df.columns = ['map_numerator', 'map_denominator']
    ap_df['ap_at_7'] = ap_df.apply(
        lambda row: row["map_numerator"] / row["map_denominator"] if row["map_denominator"] > 0 else 0,
        axis=1
    )
    return ap_df["ap_at_7"].mean()

# --- Advanced Transformer Configuration ---
class TransformerConfig:
    # Use the output from the XGBoost OOF script as input
    TRAIN_FILE = "inter/train_5_oof_xgb.parquet"
    TEST_FILE = "inter/test_5_oof_xgb.parquet"
    
    CAT_EMBED_DIM = 8
    EMBED_DIM = 192
    NUM_HEADS = 8
    NUM_LAYERS = 4
    DIM_FEEDFORWARD = 512
    DROPOUT = 0.2
    MAX_SEQ_LEN = 20
    
    EPOCHS = 15
    N_SPLITS = 5
    LR = 5e-5
    BATCH_SIZE = 256
    NUM_WORKERS = 2
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    MODEL_OUTPUT_PATH = "model/transformer_residual_model"

# --- PyTorch Dataset and Model (Unchanged from previous version) ---
class AmexAdvancedDataset(Dataset):
    def _init_(self, sequences_cat, sequences_num, labels, gbdt_preds=None):
        self.sequences_cat = torch.tensor(sequences_cat, dtype=torch.long)
        self.sequences_num = torch.tensor(sequences_num, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        if gbdt_preds is None:
            gbdt_preds = np.zeros_like(labels)
        self.gbdt_preds = torch.tensor(gbdt_preds, dtype=torch.float32)

    def _len_(self):
        return len(self.sequences_cat)

    def _getitem_(self, idx):
        return self.sequences_cat[idx], self.sequences_num[idx], self.labels[idx], self.gbdt_preds[idx]


class AmexPureTransformer(nn.Module):
    def _init_(self, config, num_numerical_features, cat_cardinalities):
        super()._init_()
        self.config = config
        self.cat_embeddings = nn.ModuleList([
            nn.Embedding(cardinality, config.CAT_EMBED_DIM) for cardinality in cat_cardinalities
        ])
        total_cat_embed_dim = len(cat_cardinalities) * config.CAT_EMBED_DIM
        self.num_norm = nn.LayerNorm(num_numerical_features)
        combined_feature_dim = total_cat_embed_dim + num_numerical_features
        self.feature_proj = nn.Linear(combined_feature_dim, config.EMBED_DIM)
        self.pos_encoder = nn.Parameter(torch.zeros(1, config.MAX_SEQ_LEN, config.EMBED_DIM))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=config.EMBED_DIM, nhead=config.NUM_HEADS,
            dim_feedforward=config.DIM_FEEDFORWARD, dropout=config.DROPOUT,
            activation='gelu', batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=config.NUM_LAYERS)
        self.head = nn.Sequential(
            nn.LayerNorm(config.EMBED_DIM),
            nn.Linear(config.EMBED_DIM, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x_cat, x_num):
        cat_embeds = [self.cat_embeddings[i](x_cat[:, :, i]) for i in range(x_cat.size(2))]
        all_cat_embeds = torch.cat(cat_embeds, dim=-1)
        x_num_norm = self.num_norm(x_num)
        x = torch.cat([all_cat_embeds, x_num_norm], dim=-1)
        x = self.feature_proj(x)
        x = x + self.pos_encoder[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.head(x)
        return x

# --- Data Preparation and Training Functions ---
def prepare_advanced_transformer_data(config):
    print("\n--- Preparing data for Advanced Transformer (Residual Modeling) ---")
    
    train_df = pd.read_parquet(config.TRAIN_FILE)
    test_df = pd.read_parquet(config.TEST_FILE)
    
    print("  -> Creating blended GBDT prediction and residual target...")
    train_df['gbdt_blend_oof'] = (train_df['oof_lgbm_prediction'] + train_df['oof_xgb_prediction']) / 2.0
    train_df['residual_target'] = train_df['y'] - train_df['gbdt_blend_oof']
    
    with open(os.path.join("model", "top_200_features.json"), 'r') as f:
        top_features = json.load(f)
    with open(os.path.join("model", "all_categorical_features.json"), 'r') as f:
        all_categorical_features = json.load(f)

    categorical_cols = [col for col in top_features if col in all_categorical_features]
    numerical_cols = [col for col in top_features if col not in categorical_cols]
    
    if 'oof_lgbm_prediction' not in numerical_cols:
        numerical_cols.append('oof_lgbm_prediction')
    if 'oof_xgb_prediction' not in numerical_cols:
        numerical_cols.append('oof_xgb_prediction')

    cat_cardinalities = []
    for col in tqdm(categorical_cols, desc="  -> Encoding Categoricals (Leakage-Free)"):
        learned_categories = train_df[col].astype('category').cat.categories
        mapping = {cat: code for code, cat in enumerate(learned_categories, 1)}
        train_df[col] = train_df[col].map(mapping).fillna(0).astype(int)
        test_df[col] = test_df[col].map(mapping).fillna(0).astype(int)
        cat_cardinalities.append(len(mapping) + 1)

    def create_sequences(df, cols_cat, cols_num, lbl_col=None, is_test=False):
        sequences_cat, sequences_num, labels, customer_ids = [], [], [], []
        grouped = df.groupby('id2')
        for cust_id, group in tqdm(grouped, desc=f"Creating sequences ({'test' if is_test else 'train'})"):
            cat_feats, num_feats = group[cols_cat].values, group[cols_num].values
            padding_len = config.MAX_SEQ_LEN - len(group)
            if padding_len > 0:
                cat_feats = np.pad(cat_feats, ((padding_len, 0), (0, 0)), 'constant', constant_values=0)
                num_feats = np.pad(num_feats, ((padding_len, 0), (0, 0)), 'constant', constant_values=0)
            else:
                cat_feats, num_feats = cat_feats[-config.MAX_SEQ_LEN:], num_feats[-config.MAX_SEQ_LEN:]
            sequences_cat.append(cat_feats)
            sequences_num.append(num_feats)
            customer_ids.append(cust_id)
            if not is_test:
                group_labels = group[lbl_col].values
                if padding_len > 0:
                    padded_labels = np.pad(group_labels, (padding_len, 0), 'constant', constant_values=-100.0)
                else:
                    padded_labels = group_labels[-config.MAX_SEQ_LEN:]
                labels.append(padded_labels)
        final_cat, final_num = np.array(sequences_cat), np.nan_to_num(np.array(sequences_num))
        if is_test:
            return final_cat, final_num, np.array(customer_ids)
        return final_cat, final_num, np.array(labels), np.array(customer_ids)

    cat_train, num_train, lbl_train, _ = create_sequences(train_df, categorical_cols, numerical_cols, 'residual_target')
    cat_test, num_test, _ = create_sequences(test_df, categorical_cols, numerical_cols, is_test=True)
    
    train_data = (cat_train, num_train, lbl_train, train_df)
    test_data = (cat_test, num_test, test_df)
    
    return train_data, test_data, len(numerical_cols), cat_cardinalities

### --- UPDATED: VECTORIZED LISTWISE LOSS FUNCTION --- ###
def train_one_epoch_listwise_vectorized(model, dataloader, optimizer, device):
    """
    Trains the model for one epoch using a fully vectorized ListNet loss calculation,
    which is significantly faster on GPU than the previous iterative approach.
    """
    model.train()
    total_loss = 0.0

    for batch in tqdm(dataloader, desc="Training (Vectorized Listwise)", leave=False):
        cat_seq, num_seq, labels, gbdt_preds = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)

        # Get model predictions for the residuals
        residual_preds = model(cat_seq, num_seq).squeeze(-1)

        # --- Vectorized Loss Calculation ---
        # Create a mask to identify non-padded elements
        mask = (labels != -100.0)
        
        # Create a batch index tensor to identify which items belong to which sequence
        # Example: [0, 0, 0, 1, 1, 2, 2, 2, 2, ...]
        batch_idx = torch.arange(labels.size(0), device=device).unsqueeze(1).expand_as(labels)
        
        # Flatten all tensors and apply the mask to keep only valid (non-padded) items
        flat_batch_idx = batch_idx[mask]
        
        # If there are no valid labels in the batch, skip to the next one
        if flat_batch_idx.numel() == 0:
            continue

        # Get the corresponding predictions and labels for the valid items
        flat_residual_preds = residual_preds[mask]
        flat_gbdt_preds = gbdt_preds[mask]
        flat_labels = labels[mask]

        # Reconstruct the final predictions and true labels in their flattened 1D form
        flat_final_preds = flat_gbdt_preds + flat_residual_preds
        flat_y_true = flat_gbdt_preds + flat_labels

        # Calculate the "ideal" probability distribution from true labels for each sequence
        # This performs a grouped softmax based on the batch index
        y_prob_dist = scatter_softmax(flat_y_true, flat_batch_idx, dim=0)
        
        # Calculate the model's predicted log-probability distribution for each sequence
        log_pred_prob_dist = scatter_softmax(flat_final_preds, flat_batch_idx, dim=0).log()

        # Calculate the KL Divergence term for each individual item
        kl_div_terms = - (y_prob_dist * log_pred_prob_dist)

        # Sum the terms for each sequence using scatter_sum to get the loss per sequence
        sequence_losses = scatter_sum(kl_div_terms, flat_batch_idx, dim=0)

        # Calculate the mean loss over all sequences in the batch
        mean_loss = sequence_losses.mean()

        # --- Standard PyTorch optimization step ---
        optimizer.zero_grad()
        mean_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += mean_loss.item()
        
    return total_loss / len(dataloader)


def predict_transformer(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting", leave=False):
            outputs = model(batch[0].to(device), batch[1].to(device)).squeeze(-1)
            # Create a mask to filter out predictions for padded items
            mask = (batch[0][:, :, 0] != 0).to(device) 
            for i in range(outputs.shape[0]):
                # Only keep predictions for the original, non-padded sequence items
                predictions.extend(outputs[i][mask[i]].cpu().numpy())
    return np.array(predictions)

def evaluate_model(model, dataloader, device, ids_df, gbdt_blend_oof_fold):
    model.eval()
    residual_preds = predict_transformer(model, dataloader, device)
    eval_df = ids_df.copy()
    
    if len(eval_df) != len(residual_preds):
        print(f"CRITICAL ERROR: Length mismatch. Events: {len(eval_df)}, Preds: {len(residual_preds)}")
        return 0.0

    eval_df['residual_pred'] = residual_preds
    eval_df['gbdt_blend_pred'] = gbdt_blend_oof_fold
    eval_df['final_pred'] = eval_df['gbdt_blend_pred'] + eval_df['residual_pred']
    
    score = calculate_map_at_7(eval_df['y'].values, eval_df['final_pred'].values, eval_df[['id2', 'id5']])
    return score

def run_stage2_transformer():
    print("\n" + "="*80)
    print("🚀 === SCRIPT 2: RESIDUAL TRANSFORMER PIPELINE (VECTORIZED LISTWISE RANKING LOSS) ===")
    print("="*80)

    config = TransformerConfig()
    os.makedirs(os.path.dirname(config.MODEL_OUTPUT_PATH), exist_ok=True)

    (train_data, test_data, num_numerical, cat_cards) = prepare_advanced_transformer_data(config)
    cat_train_full, num_train_full, lbl_train_full, original_train_df = train_data
    cat_test, num_test, original_test_df = test_data

    # Group by customer ID to prepare for stratified k-fold splitting
    customer_y_stratify = original_train_df.groupby('id2')['y'].max()

    print("\n--- Creating GBDT prediction sequences for ranking loss ---")
    padded_gbdt_preds = []
    for cust_id in tqdm(original_train_df['id2'].unique(), desc="Padding GBDT Preds"):
        cust_preds = original_train_df[original_train_df['id2'] == cust_id]['gbdt_blend_oof'].values
        padding_len = config.MAX_SEQ_LEN - len(cust_preds)
        if padding_len > 0:
            padded_preds = np.pad(cust_preds, (padding_len, 0), 'constant', constant_values=0)
        else:
            padded_preds = cust_preds[-config.MAX_SEQ_LEN:]
        padded_gbdt_preds.append(padded_preds)
    gbdt_preds_seq = np.array(padded_gbdt_preds)

    skf = StratifiedKFold(n_splits=config.N_SPLITS, shuffle=True, random_state=42)
    oof_residual_preds = np.full(len(original_train_df), np.nan)
    test_preds_list = []

    for fold, (train_cust_idx, val_cust_idx) in enumerate(skf.split(np.zeros(len(customer_y_stratify)), customer_y_stratify)):
        print(f"\n{'='*30} FOLD {fold+1}/{config.N_SPLITS} {'='*30}")

        # Get the actual customer IDs for the validation fold
        val_cust_ids_fold = customer_y_stratify.index[val_cust_idx]
        # Create masks to select all events belonging to these customers
        val_events_mask = original_train_df['id2'].isin(val_cust_ids_fold)
        val_events_indices = original_train_df[val_events_mask].index

        # Split sequence data by customer indices
        X_cat_train, X_num_train, y_res_train = cat_train_full[train_cust_idx], num_train_full[train_cust_idx], lbl_train_full[train_cust_idx]
        X_cat_val, X_num_val, y_res_val = cat_train_full[val_cust_idx], num_train_full[val_cust_idx], lbl_train_full[val_cust_idx]
        X_gbdt_train, X_gbdt_val = gbdt_preds_seq[train_cust_idx], gbdt_preds_seq[val_cust_idx]
        
        # Prepare data for evaluation
        ids_val_fold_df = original_train_df.loc[val_events_indices]
        gbdt_blend_oof_val_fold = (ids_val_fold_df['oof_lgbm_prediction'] + ids_val_fold_df['oof_xgb_prediction']) / 2.0

        loader_train = DataLoader(AmexAdvancedDataset(X_cat_train, X_num_train, y_res_train, X_gbdt_train), batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS)
        loader_val = DataLoader(AmexAdvancedDataset(X_cat_val, X_num_val, y_res_val, X_gbdt_val), batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)

        model = AmexPureTransformer(config, num_numerical, cat_cards).to(config.DEVICE)
        optimizer = AdamW(model.parameters(), lr=config.LR, weight_decay=1e-2)
        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)

        best_map_score = -1.0
        patience_counter = 0
        
        for epoch in range(config.EPOCHS):
            # --- MODIFIED: Call the new vectorized training function ---
            train_loss = train_one_epoch_listwise_vectorized(model, loader_train, optimizer, config.DEVICE)
            
            map_score = evaluate_model(model, loader_val, config.DEVICE, ids_val_fold_df, gbdt_blend_oof_val_fold.values)
            print(f"Epoch {epoch+1}/{config.EPOCHS} -> Train Loss: {train_loss:.6f} | Val MAP@7: {map_score:.6f}")
            
            scheduler.step(map_score)

            if map_score > best_map_score:
                best_map_score = map_score
                torch.save(model.state_dict(), f"{config.MODEL_OUTPUT_PATH}fold{fold+1}.pth")
                print(f"  -> 🎉 New best score! Model for Fold {fold+1} saved.")
                patience_counter = 0
            else:
                patience_counter += 1
            
            if patience_counter >= 4:
                print("  -> Stopping early due to lack of improvement.")
                break

        print(f"-> Loading best model for Fold {fold+1} (MAP@7: {best_map_score:.6f}) for final predictions...")
        model.load_state_dict(torch.load(f"{config.MODEL_OUTPUT_PATH}fold{fold+1}.pth"))
        
        # Predict on the validation part of this fold to generate OOF predictions
        val_preds_fold = predict_transformer(model, loader_val, config.DEVICE)
        oof_residual_preds[val_events_indices] = val_preds_fold
        
        # Predict on the full test set
        test_loader = DataLoader(AmexAdvancedDataset(cat_test, num_test, np.zeros((len(cat_test), config.MAX_SEQ_LEN))), batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS)
        test_preds_fold = predict_transformer(model, test_loader, config.DEVICE)
        test_preds_list.append(test_preds_fold)
        
        del model, loader_train, loader_val; gc.collect(); torch.cuda.empty_cache()

    print("\n--- K-Fold Training Finished. Saving final OOF and Test predictions. ---")
    
    final_train_df = pd.read_parquet(config.TRAIN_FILE)
    final_train_df['oof_transformer_residual'] = oof_residual_preds
    final_train_df.to_parquet("inter/train_6_final_ensemble.parquet", index=False)
    print("✅ Saved 'train_6_final_ensemble.parquet' with Transformer OOF residual predictions.")
    
    avg_test_preds = np.mean(test_preds_list, axis=0)
    final_test_df = pd.read_parquet(config.TEST_FILE)
    final_test_df['transformer_residual'] = avg_test_preds
    final_test_df.to_parquet("inter/test_6_final_ensemble.parquet", index=False)
    print("✅ Saved 'test_6_final_ensemble.parquet' with averaged Transformer test residual predictions.")

if _name_ == "_main_":
    warnings.filterwarnings("ignore", category=UserWarning)
    # Run the Transformer stage
    run_stage2_transformer()

# =================================================================================
#   SCRIPT 3: FINAL ENSEMBLING AND SUBMISSION (NO CHANGES NEEDED)
# =================================================================================
def run_final_ensembling():
    print("\n" + "="*80)
    print("🚀 === SCRIPT 3: FINAL LOGISTIC ENSEMBLING & SUBMISSION ===")
    print("="*80 + "\n")
    
    try:
        train_full = pd.read_parquet("inter/train_6_final_ensemble.parquet")
        test_full = pd.read_parquet("inter/test_6_final_ensemble.parquet")
        print(f"Train data shape: {train_full.shape}")
        print(f"Test data shape: {test_full.shape}")
    except FileNotFoundError as e:
        print(f"❌ ERROR: Could not find an input file. Please ensure all previous model stages have run. Details: {e}")
        return

    print("\n--- Training Logistic Regression to find optimal blending weights ---")
    
    ensemble_features = ['oof_lgbm_prediction', 'oof_xgb_prediction', 'oof_transformer_residual']
    X_ensemble_train = train_full[ensemble_features].fillna(0)
    y_ensemble_train = train_full['y']

    ensemble_model = LogisticRegression(class_weight='balanced', C=0.1, solver='liblinear')
    ensemble_model.fit(X_ensemble_train, y_ensemble_train)
    
    print("✅ Ensemble model trained.")
    print(f"Learned Coefficients (Weights):")
    for feature, coef in zip(ensemble_features, ensemble_model.coef_[0]):
        print(f"  - {feature}: {coef:.4f}")

    print("\n--- Generating final predictions on the test set ---")
    
    # Select the same feature columns from the test set for prediction
    X_ensemble_test = test_full[ensemble_features].copy().fillna(0)

    final_predictions_prob = ensemble_model.predict_proba(X_ensemble_test)[:, 1]

    print("\n--- Creating final submission file ---")
    
    submission_df = test_full[['id1', 'id2', 'id3', 'id5']].copy()
    submission_df['pred'] = final_predictions_prob
    submission_df['id5'] = pd.to_datetime(submission_df['id5']).dt.date
    
    output_path = 'final_ensemble_submission.csv'
    submission_df.to_csv(output_path, index=False)
    
    print(f"\n🎉🎉 Final ensemble submission file created successfully: '{output_path}' 🎉🎉")
    print("Submission Head:")
    print(submission_df.head())

if _name_ == "_main_":
    warnings.filterwarnings("ignore", category=UserWarning)
    # Run the final ensembling and create submission
    run_final_ensembling()

# =================================================================================
# 🚀 SCRIPT 4: FEATURE IMPORTANCE REPORT (NO CHANGES NEEDED)
# =================================================================================
def generate_feature_importance_report():
    print("\n" + "="*80)
    print("🚀 === SCRIPT 4: Generating Feature Importance Report ===")
    print("="*80 + "\n")

    MODEL_DIR = "model"
    N_SPLITS = 5
    all_importances = []
    
    print(f"--- Loading {N_SPLITS} fold models to average importance ---")
    
    try:
        with open(os.path.join(MODEL_DIR, "top_200_features.json"), "r") as f:
            top_features = json.load(f)

        for fold in range(1, N_SPLITS + 1):
            model_path = os.path.join(MODEL_DIR, f"lgbm_ranker_fold_{fold}.txt")
            model = lgb.Booster(model_file=model_path)
            importance_gain = model.feature_importance(importance_type='gain')
            fold_importance_df = pd.DataFrame({
                'feature': top_features,
                f'importance_gain_fold_{fold}': importance_gain
            })
            all_importances.append(fold_importance_df)

    except FileNotFoundError as e:
        print(f"❌ Error: Could not find a model file. Details: {e}")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return

    if not all_importances:
        print("No importance data found.")
        return

    final_importance_df = all_importances[0]
    for i in range(1, len(all_importances)):
        final_importance_df = pd.merge(final_importance_df, all_importances[i], on='feature', how='outer')

    gain_cols = [col for col in final_importance_df.columns if 'importance_gain' in col]
    final_importance_df['average_gain_importance'] = final_importance_df[gain_cols].mean(axis=1)

    report_df = final_importance_df[['feature', 'average_gain_importance']].sort_values(
        by='average_gain_importance', ascending=False
    ).reset_index(drop=True)

    output_path = 'feature_importance_report.xlsx'
    report_df.to_excel(output_path, index=False, engine='openpyxl')
    
    print(f"\n✅ Feature importance report saved successfully to '{output_path}'")
    print("--- Top 10 Most Important Features ---")
    print(report_df.head(10))

# =================================================================================
# --- MAIN EXECUTION BLOCK ---
# =================================================================================
if _name_ == "_main_":
    warnings.filterwarnings("ignore", category=UserWarning)
    # Generate the feature importance report from the GBDT models
    generate_feature_importance_report()